# DATASCI 290 - GenAI - Assignment 5

In Assignment 5 you will create and test a RAG system yourself, and write a corresponding business proposal.

The overall scenario is as follows:

You work at a tech company that is looking for new ways to organize their question answering and search capabilities to accelerate both engineering activity and the marketing team. The company also wants to roll out new GenAI-based products, so a lot of the questions will center around Generative AI concepts. The company has about 300 engineers and a marketing staff of 40. Product releases are done quarterly.

Your role is to implement and conduct a (mini-)POC helping the company to evaluate RAG capabilities for the improvement of their document search (and corresponding question answering), supporting particularly the engineering and marketing organizations. You will have a gold dataset with 'good' responses to questions from marketing and engineering teams. You need to develop metric(s) that help you to evaluate how well your RAG system performs relative to the gold data. You should work with the tunables of the setup (LLM, chunking, embeddings, ...) for your iterations.

You will also need to write up your findings as a short proposal.

(See instructions throughout this notebook.)

So overall, the goals of this assignment is for you to:

*  To implement a RAG system using LangChain
*  Be able to formulate metric(s) that you may want to choose as your evaluation to what degree your system replicates gold answers (labeled data) that we will provide.
* Try out various hyper-parameters and settings to see which configuration works the best (given your chosen metric)  
* Write a comprehensive evaluation, which also includes risks and limitations (and a lot more)

The notebook is organized as follows:

1. Set-Up

2. Base RAG components

    We will provide a base LangChain-based framework for you to use for your RAG system. The components we’ll need include:  

  2.1 Text Embeddings    
  2.2 Text Chunking   
  2.3 The Vector DB & Semantic Search  
  2.4 The Language Model   
  2.5 Testing the LLM in a LangChain Chain   
  2.6. Setting up a simple RAG Chain     


3. Using RAG  
  3.1 Loading of Data  
  3.2 Test Queries


4.  Evaluations

  Here, you will conduct your evaluations


5. Final Results

  In this section you provide the RAG answers to the test questions

RULES:  

* You can only use the language models specified here  
* You can only use the embedding methods we discuss  
* You can only use the focuments we provide. And they all must be in your store   
* Apart from the provided specifications, some of the things you can freely experiment with include chunk sizes, prompts, etc.


**To run this notebook** you should copy it to your personal Colab Pro Google account by uploading it into your Google Drive. From there you can open it as a Colab notebook and run it.  Note it needs a T4 GPU to run.  You may be able to run it in a free Colab notebook.

NOTES:
* The Open Source Model is not trained for safety. So unsafe answers could be returned.


Let's begin!

## 1. Setup

We will first install a number of libraries and import what we will need.






In [ ]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install einops
!pip install faiss-gpu
!pip install langchain_community
!pip install --upgrade --quiet chromadb bs4 qdrant-client
!pip install langchainhub
!pip install -U langchain-huggingface
!pip install -U langchain-cohere
!pip install --upgrade --quiet  wikipedia
!pip install --upgrade --quiet  arxiv
!pip install --upgrade --quiet  pymupdf

!pip install xmltodict

!pip install cohere

!pip install git+https://github.com/Tiiiger/bert_score

In [ ]:
import torch
import os
import bs4
import json
import numpy as np
import time


from pprint import pprint

import locale

from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers import pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain.llms import HuggingFacePipeline
from langchain_cohere import ChatCohere
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.utils.math import cosine_similarity

from langchain_community.document_loaders import ArxivLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PubMedLoader

#from langchain_community.chat_models import ChatCohere

from google.colab import userdata

from google.colab import drive
import re
import pandas as pd
from bert_score import BERTScorer
import torch

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%%capture
!pip install sentence_transformers

Add your keys from the secret store (do **NOT** print them out or leave them exposed as plaintext in your notebook!):

In [ ]:
COHERE_API_KEY = userdata.get('COHERE_API_KEY')

## 2. Building the Components of our RAG System

Let us introduce and test the base components of our RAG system. We will largely use the Hugging Face and LangChan libraries.



### 2.1 The Embedding Model

We will need to represent text (pieces) as vectors. For this, we will use the [sentence_transformer]() architecture.



**NOTE:** The models you can use are: 'all-mpnet-base-v2', 'all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1', 'all-distilroberta-v1', and 'avsolatorio/GIST-Embedding-v0'



In [ ]:
%%capture
# embedding model to represent the text as vectors
base_embeddings = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

In [ ]:
# define the text for the query
text = "This is a test document."

# query text to embedding
query_result = base_embeddings.embed_query(text)
print(f'Embedding dimension: {len(query_result)}')

# define the two documents and convert both of them to embeddings
doc_result = base_embeddings.embed_documents(["Germany won the World Cup 4 times.", "This is not a test document."])
len(doc_result)

Embedding dimension: 768


2

Do those dimensions look correct?

Now lets see if the embedding model is working as we want.  Ideally our embeddings go beyond shared words and capture the underlying meaning.

In [ ]:
#Let's see how well our embeddng model works

# define the cosine similarity between the query and the two documents
similarity = cosine_similarity([query_result], doc_result)[0]

similarity

array([0.21410193, 0.94317828])

That's how you should define your embedding models.

Next, we turn to text chunks.

### 2.2. Loading and Chunking Texts

We first need to load the documents. Here is an example:

In [ ]:
# load the documents (github article about agents)
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

documents = loader.load()

We will need to split the  text in chunks that are 'suitable' as retrieval units. Let's for starters define a chunk size of 128 and have no overlap between the chunks:  


In [ ]:
# split the github article into chunks that do not overlap, such that sizes of chunks are all 128.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=0)
splits = text_splitter.split_documents(documents)
print('Number of splits/chunks: ', str(len(splits)))

Number of splits/chunks:  444


Ok, so it looks like we have now many splits (chunks) from one document. Here is how you can get the content:

In [ ]:
# print one of the chunks
splits[50].page_content

'reasoning skills. Reflexion has a standard RL setup, in which the reward model provides a simple binary reward and the action'

Perfect. Now we have the splits and embeddings. Next, the embeddings need to be stored in a vector db.

### 2.3 Storing the Embeddings of Chunks in Vectorstores

After loading and chunking the data, we need to save the vector representations of the chunks in a vectorstore. We will use Qdrant here for simplicity. We load the splits (structured chunks) and the embeddings:

In [ ]:
# saving vector representations of chunks into vectorstore using Qdrant
vectorstore = Qdrant.from_documents(splits, # the chunks we just split
    base_embeddings, # using the hugging face embedding model
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="test",
)
retriever = vectorstore.as_retriever()

The nice thing is that the vector store also does the similarity searches for us:

In [ ]:
# using the vectorstore (into which we just saved the chunks from the article) to do a similarity search to rank the most highly similar chunks
query = "What is Chain of Thought doing?"
docs = vectorstore.similarity_search_by_vector(base_embeddings.embed_query(query)) # will rank the splits

In [ ]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': '6be55c16773944f49ee22af155702aff', '_collection_name': 'test'}, page_content='the model’s thinking process.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': '7a5e2d1556484bd09b9c53522616ca15', '_collection_name': 'test'}, page_content='[1] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': 'a99ecdf9838c4794949f845e918f39b2', '_collection_name': 'test'}, page_content='the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': 'c8921b7cb8e4461a9d0110696f64b6a5', '_collection_name': 'test'}, page_content='Chain of thought (CoT; Wei et al. 2022) has 

Looks good! We have an ordered list of documents that seem to relate to the question. That is what we need.

The last major component is the actual LLM.

### 2.4. The LLM

We will use one Open Source Model ("mistralai/Mistral-7B-Instruct-v0.1") and one Proprietery Model (Cohere) for our tests. Let's first set up the OS model:

In [ ]:
%%capture
# quantization to reduce precisions of the weights of mistral
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         )

# loading the model with the specified quantization
llm_mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

# loading the tokenizer
llm_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

We use the model first to generate a Hugging Face pipeline. A pipeline simplifies the process of actually generating responses.

In [ ]:
# creating a text generation pipeline
mistral_pipe = pipeline(
    "text-generation",
    model=llm_mistral_model, # specifying the pre-trained model to use for text generation
    tokenizer=llm_mistral_tokenizer, # specifying the tokenizer
    max_new_tokens=1000, # sets the maximum number of new tokens to generate.
    temperature=0.6, # controls the randomness of predictions
    top_p=0.95, # generates more coherent text
    do_sample=True, # enables sampling
    repetition_penalty=1.2 # penalizes repeated tokens to avoid redundancy in the generated text.
)

# setting the padding token ID
mistral_pipe.model.config.pad_token_id = mistral_pipe.model.config.eos_token_id

    # wrapping the Hugging Face pipeline into a LangChain object

Does it work?

In [ ]:
mistral_pipe("[INST]Give me a two-sentence story about an apple![/INST]")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '[INST]Give me a two-sentence story about an apple![/INST]Once upon a time, in the heart of an enchanted orchard, an apple hung from a tree, its radiant red skin shimmering under the sun. A young girl, with eyes full of wonder and dreams, reached up to pluck it, her heart filled with anticipation for the sweet, juicy treasure that awaited her within.'}]

Reasonable!

We will also use a Cohere model, but will create this below as part of the LangChain framework.

### 2.5 Testing the LLM in a LangChain Chain

Chains will be defined and discussed in Week 11. In short, they are convenient programmatic ways to deal with 'chains' of actions that involve LLMs. For example, a list of events like 'here is a city name. Plug that city name into prompt template, then generate a story about that city. Lastly, format the model output as a string' can be easily handled by LangChain's Chain framework. In this case, the Chain would consist of the prompt template, the LLM, and the String Formatter. The parameter (the city in this case) will be provided at run time by invocation of the Chain. Let's test that.

To use a Hugging Face model in a LangChain environment, we need to wrap the model into a LangChain pipeline object:

In [ ]:
# wrapping the hugging face model in a lang chain pipeline object
mistral_llm_lc = HuggingFacePipeline(pipeline=mistral_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


Next, we need to define a template and create a corresponding prompt template that can take any questiion

In [ ]:
# setting the template
test_llm_template = """[INST] Give me a two-sentence story about an {object}! [/INST]"""

# setting the prompt template and specifying the input variable "object"
test_llm_prompt_template = PromptTemplate(template=test_llm_template, input_variables=["object"])



Let's define a Chain, a static flow of actions that (usually) involve at least a definition of the variables used in the chain, one or more templates, LLM step(s) and potentially other actions. This would be a chain that declares the variable 'object' to be expected when the chain is invoked, then inserts it into the template, and passes this to our mistral model pipeline (wrapped as a LangChain object):    

In [ ]:
# defining the chain which involves the definition of the "object" input variable,
# the test LLM prompt template, and the langchain version of the hugging face mistral
# LLM pipeline

test_llm_chain_short = (
    {"object": RunnablePassthrough()}
    | test_llm_prompt_template
    | mistral_llm_lc
)

In [ ]:
# invoking the chain with the input
test_llm_chain_short.invoke('apple')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'[INST] Give me a two-sentence story about an apple! [/INST] In the heart of an old orchard, an apple hung ripe and red on a weathered tree branch. A young boy climbed up with eager eyes, plucking it to share its sweetness with his grandmother.'

Works too. We will use this notation moving forward.

Next, how would we do this with a Cohere Chat Model instead of Mistral?

In [ ]:
# defining a cohere chat model
cohere_chat_model = ChatCohere(cohere_api_key=COHERE_API_KEY)

This can be plugged straight into the Chain:

In [ ]:
# plugging the cohere chat model into the chain instead of the mistral one
test_cohere_llm_chain_short = (
    {"object": RunnablePassthrough()}
    | test_llm_prompt_template
    | cohere_chat_model # replaced with cohere
)

In [ ]:
test_cohere_llm_chain_short.invoke('apple')

AIMessage(content='The apple, once bitter and unloved, fell from the tree and, over time, ripened into something sweet and delicious. Its journey from tree to ground transformed it into something cherished and desired by all.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c4ff1794-4c65-4516-aed2-f5a2264769df', 'token_count': {'input_tokens': 83, 'output_tokens': 42}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c4ff1794-4c65-4516-aed2-f5a2264769df', 'token_count': {'input_tokens': 83, 'output_tokens': 42}}, id='run-81524fce-4a95-4286-9e6d-18d1dbce7ed7-0', usage_metadata={'input_tokens': 83, 'output_tokens': 42, 'total_tokens': 125})

Works! (Note: you may want to review the format of the template. The one we used here is the one from Mistral, and the format may or may not be optimal for Cohere.)

How can we get the output formatting under control? We can add a String Formatter to the chain:


In [ ]:
# langchain output parser turns the LLM output into the output string
output_parser = StrOutputParser()

# rerunning the llm chain with the output parser
test_cohere_llm_chain_short_formatted = (
    {"object": RunnablePassthrough()}
    | test_llm_prompt_template
    | cohere_chat_model
    | output_parser
)

test_cohere_llm_chain_short_formatted.invoke('apple')

'The apple, once bitter and unloved, fell from the tree and transformed into a sweet, juicy delight. Its once tart flesh now offered a burst of flavor, a gift to those who found it.'

### 2.6 Setting Up a Simple RAG Chain

For RAG, we will follow the same approach. Except... you will **later** need to change the chain to include the retrieval step.

We first do a simple test: create a RAG template that takes a question and a pre-defined context as input, and generates the answer based on the provided context:

In [ ]:
rag_template = """[INST] Answer the question based only on the following context:
{context}

Question: {question}
[/INST]
"""
rag_prompt_template = ChatPromptTemplate.from_template(rag_template)

base_rag_chain =(
    {"context": RunnablePassthrough(),
     "question": RunnablePassthrough()}
    | rag_prompt_template
    | mistral_llm_lc
    | output_parser
)

predefined_context = "Germany has won the World Cup 4 times."
question = "How many times did Germany win the world cup?"

resp = base_rag_chain.invoke({'context': predefined_context,
                           'question': question})
print(resp)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: [INST] Answer the question based only on the following context:
{'context': 'Germany has won the World Cup 4 times.', 'question': 'How many times did Germany win the world cup?'}

Question: {'context': 'Germany has won the World Cup 4 times.', 'question': 'How many times did Germany win the world cup?'}
[/INST]
Answer: The answer to your question is consistent with the information provided in the context. According to the context, Germany has won the World Cup a total of 4 times.


That's great. But of course, the context needs to be created in an earlier retrieval step. More precisely, the documents will be first retrieved as a list, and then they will need to be formatted into one string to pass to the LLM in the context window.

Here is a simple formatting function that can be hooked into the chain, which combines a list of chunks into one string:



In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

So how could we build a simple chain? Let's first just get the retrieval done and the formatted retrieved data and the question inserted into the prompt template:

In [ ]:
rag_template = """Here is a context:\n{context} \n\nand here is a question: \n{question}"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

rag_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}
    | rag_prompt


)

In [ ]:
output = rag_chain.invoke('What is Chain of Thought?')

Ok... with some formatting... this looks good:

In [ ]:
print(output.messages[0].content)

Here is a context:
the model’s thinking process.

[1] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022

the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes 

and here is a question: 
What is Chain of Thought?


Let's complete the RAG Chain:

In [ ]:
output_parser = StrOutputParser()

rag_template = """[INST]Please answer the question below only based on the context information provided.\n\nHere is a context:\n{context} \n\nHere is a question: \n{question}.[/INST]"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

rag_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}
    | rag_prompt
    | mistral_llm_lc

)



In [ ]:
rag_chain.invoke('What is Chain of Thought?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Human: [INST]Please answer the question below only based on the context information provided.\n\nHere is a context:\nthe model’s thinking process.\n\n[1] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022\n\nthe problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process\n\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes \n\nHere is a question: \nWhat is Chain of Thought?.[/INST] According to the context provided, "Chain of Thought" (CoT) refers to a method or framework proposed by Wei et al. for eliciting reasoning from large language models. In this approach, the problem is broken down into multiple thought steps, and the model generates multiple thoughts per step. This results in a tree-like structure representing the model\'s line of reasoning.'

What about the Cohere models?

In [ ]:
cohere_rag_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}
    | rag_prompt
    | cohere_chat_model
    | output_parser
)

In [ ]:
cohere_rag_chain.invoke('What is Chain of Thought?')

'Chain of Thought (CoT) is a prompting technique used to elicit reasoning and improve the performance of large language models on complex tasks. It involves decomposing a problem into multiple thought steps and generating a sequence of thoughts or reasoning chains to arrive at an answer. By providing intermediate steps and thoughts, CoT helps the language model to break down a complex task into simpler sub-tasks, enhancing its ability to provide justified and explainable responses.'

Works too! Time to build the real thing and do experimentation.

## 3. The RAG Model & Experimentation

With this we can get started. First, we need to acquire the data, chunk it, vectorize it, and store the embeddings (and in this simple case also the docs) in our Qdrant vector db.

### 3.1 The Vector Database

We will start by creating our datastore, Qdrant. Usually, you would deploy the vector db as a server, but in this case let's simply put everything in memory. Also, in this case we will store not only the embeddings but the whole document in the vector store. We will seed the store with the splits from the blog post we had used before.

We will also create the retriever, which defines the way the documents are being retrieved. The retriever parameters define for example which method is used, how many docs are retrieved, etc. See [this LangChain link ](https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore)for more information.


In [ ]:
qdrant_vectorstore = Qdrant.from_documents(splits,
    base_embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="rag_tech_db",
    force_recreate=True
)

retriever = qdrant_vectorstore.as_retriever()

### 3.2 Data Acquisition, Chunking, and Vectorization

Now where we have our store we need to get the data into it. We will need to retrieve the data, create the chunks, then vectorize them, and finally store the vectors (along with the docs in this case) in the vector db.

Let us first set chunk size and overlap, as well as the type of splitter. These are starting parameters and you may want to experiment with them:

In [ ]:
#Note that these defaults may or may not be ideal!
CHUNK_SIZE=128
OVERLAP=0

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=OVERLAP)

Now let's work with an actual document collection.  We will work with four types of documents:

* A few papers from the ArXiv on RAG and NLP
* A few blogs from Lily Weng that talk about Open Domain Question Answering and related topics
* A number of Wikipedia articles on that topic

To make testing easier  we'll define a global record number so we can trace back to see which chunk came from which specific document.


In [ ]:
#assign a unique number to each document we ingest
global_doc_number = 1

First we'll grab some papers from ArXiv.  We'll grab the pdf files and get all of the pages as separate documents.

In [ ]:
arxiv_numbers = ('2005.11401', '2104.07567', '2104.09864', '2105.03011', '2106.09685', '2203.02155', '2211.09260', '2211.12561',
                 '2212.09741', '2305.14314', '2305.18290', '2306.15595', '2309.08872', '2309.15217', '2310.06825', '2310.11511',
                 '2311.08377', '2312.05708', '2401.06532', '2401.17268', '2402.01306', '2402.19473', '2406.04744')

In [ ]:
all_arxiv_pages = []

#loop through the papers
for identifier in arxiv_numbers:
    # Construct URL using the arXiv unique identifier
    arx_url = f"https://arxiv.org/pdf/{identifier}.pdf"

    # Extract pages from the document and add them to the list of pages
    arx_loader = PyMuPDFLoader(arx_url)
    arx_pages = arx_loader.load()
    for page_num in range(len(arx_pages)):
        page = arx_pages[page_num]
        #CHANGED
        page.metadata['page_num'] = page_num
        page.metadata['doc_num'] = global_doc_number
        page.metadata['doc_source'] = "ArXiv"
        all_arxiv_pages.append(page)


    global_doc_number += 1

How many docs did we get?  Is that the correct number? And what is the content?

In [ ]:
num_pages = len(all_arxiv_pages)
num_docs = global_doc_number - 1

print(f"{num_docs} documents in total")
print(f"{num_pages} pages in total")

23 documents in total
485 pages in total


In [ ]:
all_arxiv_pages[5].page_content[:150]  # all pages of the Document content

'Table 1: Open-Domain QA Test Scores. For TQA,\nleft column uses the standard test set for Open-\nDomain QA, right column uses the TQA-Wiki\ntest set. See'

Now we need to split the docs into chunks.  LangChain provides a couple of ways to do that.  We'll use for now the `RecursiveCharacterTextSplitter`.

In [ ]:
#index doc chunks
splits = text_splitter.split_documents(all_arxiv_pages)
for idx, text in enumerate(splits):
    splits[idx].metadata['split_id'] = idx

print('Number of splits/chunks: ', len(splits))

Number of splits/chunks:  17515


In [ ]:
splits[0]

Document(metadata={'source': 'https://arxiv.org/pdf/2005.11401.pdf', 'file_path': 'https://arxiv.org/pdf/2005.11401.pdf', 'page': 0, 'total_pages': 19, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationDate': 'D:20210413004838Z', 'modDate': 'D:20210413004838Z', 'trapped': '', 'page_num': 0, 'doc_num': 1, 'doc_source': 'ArXiv', 'split_id': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,')

Let's add the vectors to the datastore and see whether we can retrieve a nearest neighbor to a query. Let's look at the second closest match:

In [ ]:
%%capture

qdrant_vectorstore.add_documents(documents=splits)

In [ ]:
query = "How can we train a model for preferences?"
found_docs = qdrant_vectorstore.similarity_search_with_score(query)

In [ ]:
print(found_docs[0][0].page_content)
print(found_docs[0][1])

One path forward could be to train models that can be conditioned on the preferences of certain
0.8218969669738703


Next, let's get some information from Wikipedia on our main topic -- Gen AI.  LangChain provides a DocumentLoader that accesses the Wikipedia API.

In [ ]:
wiki_docs = WikipediaLoader(query="Generative Artificial Intelligence", load_max_docs=4).load()
for idx, text in enumerate(wiki_docs):
    wiki_docs[idx].metadata['doc_num'] = global_doc_number
    wiki_docs[idx].metadata['doc_source'] = "Wikipedia"

global_doc_number += 1

print('Number of documents: ', len(wiki_docs))

#index docs
wiki_splits = text_splitter.split_documents(wiki_docs)
for idx, text in enumerate(wiki_splits):
    wiki_splits[idx].metadata['split_id'] = idx

print('Number of splits/chunks: ', len(wiki_splits))


Number of documents:  4
Number of splits/chunks:  151


Now we'll add these splits to the vector stores.

In [ ]:
%%capture

#vectorstore.add_documents(documents=wiki_splits, embedding=base_embeddings)
qdrant_vectorstore.add_documents(documents=wiki_splits)

Same with a couple of other queries:

In [ ]:
wiki_docs = WikipediaLoader(query="Information Retrieval", load_max_docs=4).load()
for idx, text in enumerate(wiki_docs):
    wiki_docs[idx].metadata['doc_num'] = global_doc_number
    wiki_docs[idx].metadata['doc_source'] = "Wikipedia"

global_doc_number += 1

print('Number of documents: ', len(wiki_docs))

#index docs
wiki_splits = text_splitter.split_documents(wiki_docs)
for idx, text in enumerate(wiki_splits):
    wiki_splits[idx].metadata['split_id'] = idx

print('Number of splits/chunks: ', len(wiki_splits))

Number of documents:  4
Number of splits/chunks:  160


In [ ]:
%%capture

#vectorstore.add_documents(documents=wiki_splits, embedding=base_embeddings)
qdrant_vectorstore.add_documents(documents=wiki_splits)

And yet another related Wikipedia article.

In [ ]:
wiki_docs = WikipediaLoader(query="Large Language Models", load_max_docs=4).load()
for idx, text in enumerate(wiki_docs):
    wiki_docs[idx].metadata['doc_num'] = global_doc_number
    wiki_docs[idx].metadata['doc_source'] = "Wikipedia"

global_doc_number += 1

print('Number of documents: ', len(wiki_docs))

#index docs
wiki_splits = text_splitter.split_documents(wiki_docs)
for idx, text in enumerate(wiki_splits):
    wiki_splits[idx].metadata['split_id'] = idx

print('Number of splits/chunks: ', len(wiki_splits))

Number of documents:  4
Number of splits/chunks:  134


In [ ]:
%%capture

#vectorstore.add_documents(documents=wiki_splits, embedding=base_embeddings)
qdrant_vectorstore.add_documents(documents=wiki_splits)

We'll also augment our collection with some blog entries about Open Domain Question Answering, of which RAG is an approach, and some related topics in case users want to ask how the new Search system works.

In [ ]:
web_loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2020-10-29-odqa/",
               "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
               "https://lilianweng.github.io/posts/2018-06-24-attention/",
               "https://lilianweng.github.io/posts/2023-06-23-agent/",
               "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"),

    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

web_documents = web_loader.load()

for idx, text in enumerate(web_documents):
    web_documents[idx].metadata['doc_num'] = global_doc_number
    web_documents[idx].metadata['doc_source'] = "WWW"
global_doc_number += 1

print('Number of documents: ', len(web_documents))


Number of documents:  5


Again, we will split the retrieved data into chunks and add the data to the vector store:

In [ ]:
web_splits = text_splitter.split_documents(web_documents)

for idx, text in enumerate(web_splits):
    web_splits[idx].metadata['split_id'] = idx

print('Number of splits: ', len(web_splits))

Number of splits:  2103


In [ ]:
%%capture

qdrant_vectorstore.add_documents(documents=web_splits)

In [ ]:
retriever = qdrant_vectorstore.as_retriever()

### 3.3 The Test Data

You will want to test the system that you (will) have built. Below we give you a validation set that you could take as labeled data (imagine, your user personas would have had these questions and deemed the answers to be good). We also will give you a test set that only contains questions. (This is the set that we will use to get a feel for how well your RAG system corresponds to our Gold model).

Here are is the gold validation set and the test questions. **DO NOT CHANGE OR DELETE!!**

In [ ]:
validation_questions_answers = {
    0: {"question": "What purpose do large language models serve in the field of natural language processing?",
  "gold_answer_research": "Large language models (LLMs) serve the purpose of enabling general-purpose language generation and other natural language processing tasks such as classification. They achieve this by learning statistical relationships from text documents during computationally intensive self-supervised and semi-supervised training. LLMs can be used for text generation by predicting the next token or word, making them valuable for tasks like speech recognition, machine translation, and information retrieval. Additionally, LLMs have superseded previous models like recurrent neural networks, showcasing their efficiency and effectiveness in NLP tasks.",
  "gold_answer_marketing": "Large language models serve the purpose of improving performance in various natural language processing tasks, such as speech recognition, machine translation, natural language generation, optical character recognition, handwriting recognition, grammar induction, and information retrieval."},
1: {"question": "How does a large language model learn from text during training?",
  "gold_answer_research": "A large language model learns from text during training by first going through an unsupervised generative 'pretraining' stage where it sets initial parameters using a language modeling objective. Then, it goes through a supervised discriminative 'fine-tuning' stage where it refines its parameters based on annotated examples or task demonstrations. This dual-stage approach allows the model to learn statistical relationships from text documents in a computationally intensive process, enabling it to achieve general-purpose language generation and natural language processing tasks.",
  "gold_answer_marketing": "A large language model learns from text during training by first pretraining on a diverse dataset to acquire general language knowledge, and then fine-tuning on specific tasks or demonstrations to adapt its parameters for more targeted performance."},
2: {"question": "What are some key architectures behind the development of large language models?",
  "gold_answer_research": "Key architectures behind the development of large language models include the use of self-attention mechanisms, such as those seen in Transformer decoders. These architectures have been applied to tasks like autoregressive language modeling and have led to the dominance of Transformer-based language models in NLP. Models like BERT and GPT-2 have further advanced this paradigm, showcasing the power of large Transformer language models in achieving state-of-the-art results across various NLP tasks. Additionally, architectures like neural-retriever-in-the-loop generative-based models have shown improvements in tasks like open-domain QA and knowledge-grounded dialogue, emphasizing the importance of consistent and engaging responses in long-form generation and multi-turn conversations.",
  "gold_answer_marketing": "Key architectures behind the development of large language models include Transformer-based models such as BERT and GPT-2, which utilize self-attention mechanisms for tasks like autoregressive language modeling and knowledge-grounded dialogue. These models have shown significant success in NLP tasks and have led to advancements in general-purpose language generation and natural language processing."},
3: {"question": "Can you name some specific large language models and the companies or organizations that have developed them?",
  "gold_answer_research": "Some specific large language models include GPT-3 by OpenAI, Chinchilla by DeepMind, and BERT by Google. OpenAI developed GPT-3, DeepMind developed Chinchilla, and Google developed BERT. These models have been significant advancements in the field of natural language processing.",
  "gold_answer_marketing": "Chinchilla by DeepMind, GPT-3 by OpenAI."},
7: {"question": "What licensing models have been adopted for the distribution of source-available language models?",
  "gold_answer_research": "Based on the provided context, it seems that licensing models for the distribution of source-available language models have not been explicitly discussed in the referenced papers. However, it is crucial to consider potential licensing options such as open-source licenses (e.g., GPL, MIT) or proprietary licenses when distributing language models to ensure legal compliance and control over usage rights. Additionally, considering the implications of different licensing models on accessibility, collaboration, and commercialization is essential for determining the most suitable approach for sharing language models with the community. Further research or consultation with legal experts may be necessary to explore specific licensing strategies for source-available language models.",
  "gold_answer_marketing": "Answer: Some organizations choose open-sourcing, while others restrict access to a few organizations with resources or offer end-to-end deployment via API."},
8: {"question": "What are language models and what is their purpose in natural language processing?",
  "gold_answer_research": "Language models are probabilistic models of natural language that help predict or correct text. Their purpose in natural language processing is to assist in various tasks such as speech recognition, machine translation, natural language generation, and information retrieval. By analyzing the performance of human subjects, language models improve the understanding and generation of human-like text.",
  "gold_answer_marketing": "Language models are probabilistic models of natural language that are used in tasks such as speech recognition, machine translation, and natural language generation in natural language processing."},
9: {"question": "How have language models evolved in terms of architecture, from the 1980s to present times?",
  "gold_answer_research": "Language models have evolved significantly in terms of architecture from the 1980s to present times. In the 1980s, the first statistical language model was proposed, leading to experiments by IBM that identified areas for improvement by observing human subjects. However, it wasn't until 2017 when the transformer architecture was introduced by Google, revolutionizing the field. This development paved the way for models like BERT in 2018, which marked a shift towards large-scale transformer-based language models. These modern architectures, based on self-attention mechanisms, have dominated the field of natural language processing, achieving state-of-the-art performance in various tasks.",
  "gold_answer_marketing": "Language models have evolved from early statistical models in the 1980s to modern transformer architectures, such as BERT and GPT-2, which use self-attention mechanisms and have become dominant in natural language processing tasks."},
11: {"question": "Can you explain how maximum entropy language models work and what the partition function signifies?",
  "gold_answer_research": "Maximum entropy language models use feature functions to encode the relationship between a word and its n-gram history, aiming to maximize reward while satisfying a KL-constrained objective. The partition function, denoted as Z(x), is crucial in normalizing the probabilities of all possible outputs given the input. It represents the sum of the exponential of the reward function over all possible output sequences, making it computationally expensive to estimate but essential for accurate modeling. The partition function ensures that the model's predicted probabilities sum up to 1, providing a foundation for effective language modeling.",
  "gold_answer_marketing": "Maximum entropy language models encode the relationship between a word and the n-gram history using feature functions. The partition function in this context represents the total probability of all possible outcomes, making it a crucial factor in determining the optimal solution for the reward maximization objective."},
12: {"question": "What is the benefit of using continuous space embeddings in recurrent neural network language models?",
  "gold_answer_research": "Continuous space embeddings in recurrent neural network language models help alleviate the curse of dimensionality by representing words as non-linear combinations of weights in the embedding space. This approach helps address the data sparsity problem caused by the exponential increase in possible word sequences with vocabulary size. By utilizing continuous space embeddings, neural networks can effectively capture semantic relationships and meaning within the language model.",
  "gold_answer_marketing": "Continuous space embeddings in recurrent neural network language models help alleviate the curse of dimensionality caused by the exponential increase in possible word sequences, reducing data sparsity issues."},
13: {"question": "What challenges do large language models face in mirroring human cognitive patterns?",
  "gold_answer_research": "Large language models face challenges in mirroring human cognitive patterns because they sometimes learn patterns that humans do not learn, while also failing to learn patterns that humans typically learn. This discrepancy suggests that the models may not be plausible cognitive models, despite matching human performance in some tasks. Further research is needed to address these limitations and improve the alignment of large language models with human cognitive patterns.",
  "gold_answer_marketing": "Large language models sometimes learn patterns that humans do not learn and fail to learn patterns that humans typically do learn."},
16: {"question": "What factors influenced the development of generative language models by Anthropic?",
  "gold_answer_research": "Several factors influenced the development of generative language models by Anthropic, including the limitations in coding, math, and reasoning capabilities of the initial version Claude, the partnerships with companies like Notion and Quora to enhance the model's capabilities, and the need to address biases, unsafe content, and ethical considerations in training data. Additionally, the reliance on supervised learning and the need for controlled generation in generative models played a role in shaping the development of Anthropic's language models.",
  "gold_answer_marketing": "Factors that influenced the development of generative language models by Anthropic include partnerships with companies like Notion and Quora, limitations in coding, math, and reasoning capabilities in initial models like Claude, and the need to address biases and unsafe content in training datasets."},
17: {"question": "What is Constitutional AI and how does it affect the functionality of AI systems?",
  "gold_answer_research": "Constitutional AI is an approach developed by Anthropic for training AI systems, particularly language models like Claude, to be harmless and helpful without relying on extensive human feedback. It involves two phases: supervised learning, where the model generates responses to prompts and self-critiques based on a set of guiding principles, and reinforcement learning, where the model is trained with AI-generated feedback according to constitutional principles. This approach enables the training of AI assistants that are both helpful and harmless, with the ability to explain objections to harmful requests, enhancing transparency and reducing the need for human supervision.",
  "gold_answer_marketing": "Constitutional AI is an approach developed by Anthropic for training AI systems, particularly language models like Claude, to be harmless and helpful without relying on extensive human feedback. It involves supervised learning and reinforcement learning phases to guide the model's responses based on a set of guiding principles (a 'constitution'). This approach aims to create AI systems that are both helpful and transparent in their decision-making process, reducing the need for constant human supervision."},
18: {"question": "How do advances in AI models impact their ability to interact with different types of data, such as images?",
  "gold_answer_research": "Advances in AI models, such as multimodal models like RA-CM3, have significantly improved their ability to interact with different types of data, such as images. These models can refer to external memory, like web data, to increase their knowledge capacity, allowing them to generate correct images from entity-rich captions. Additionally, these models can perform image editing and manually specify examples in-context for better results. The use of large language models, combined with larger datasets and neural networks, has also enhanced their performance in tasks like image generation and text generation.",
  "gold_answer_marketing": "Advances in AI models, such as multimodal models like RA-CM3, allow for better interaction with different types of data, like images, by accessing external memory for increased knowledge capacity and improving performance in tasks like image generation and image editing."},
19: {"question": "What are the potential trade-offs between AI system alignment with ethical guidelines and practical utility?",
  "gold_answer_research": "The potential trade-offs between AI system alignment with ethical guidelines and practical utility include the risk of reduced performance and usability due to stringent ethical alignment measures, as seen with Claude 2. Users may face limitations and refusal of assistance for benign requests, leading to debates over the 'alignment tax' in AI development. Balancing ethical considerations with practical functionality is crucial to ensure alignment with ethical guidelines without compromising the practical utility of AI systems. Research is needed to find a middle ground that prioritizes ethical alignment while maintaining usability and performance.",
  "gold_answer_marketing": "The potential trade-offs between AI system alignment with ethical guidelines and practical utility include balancing stringent ethical alignment that may reduce usability and performance, ensuring transparency and fairness in alignment processes, and addressing the alignment tax that may impact adoption of AI systems."},
20: {"question": "How has the token handling capacity changed between different versions of the Claude model?",
  "gold_answer_research": "The token handling capacity has increased with each new version of the Claude model. Claude Instant has a context length of 100,000 tokens, Claude 2.1 doubled this to 200,000 tokens, and Claude 3 Opus default version has a context window of 200,000 tokens but can be expanded to 1 million for specific use cases. This progression shows a trend towards handling larger amounts of text data for improved performance and capabilities.",
  "gold_answer_marketing": "The token handling capacity has increased from Claude to Claude Instant to Claude 2.1, with Claude Instant having a input context length of 100,000 tokens, Claude 2.1 having a context window of 200,000 tokens, and Claude 3 Opus having a context window of 1 million tokens."},
22: {"question": "In what ways has the Claude model's ability to self-critique and revise its responses enhanced its transparency?",
  "gold_answer_research": "The Claude model's ability to self-critique and revise its responses has enhanced its transparency by allowing for iterative improvements based on past actions and mistakes. Through self-reflection, the model can refine its output by learning from feedback and generating special tokens to signal the need for retrieval or confirm the relevance, support, or completeness of its responses. This process ensures that the model's statements about the world are truthful and accurate, ultimately increasing transparency in its decision-making and reasoning processes.",
  "gold_answer_marketing": "The Claude model's ability to self-critique and revise its responses has enhanced its transparency by allowing it to generate text informed by retrieved passages, criticize the output, and signal the need for retrieval or confirm the output's relevance, support, or completeness. This self-reflection process helps improve the model's accuracy and reliability in generating responses."},
23: {"question": "How do subsequent versions of Claude compare in terms of their likelihood to produce false statements?",
  "gold_answer_research": "Claude Instant is a faster and lighter version of Claude, with an input context length of 100,000 tokens. In contrast, Claude 3 has faced criticism for its stringent ethical alignment, leading to a debate over the 'alignment tax' in AI development. Users have been refused assistance with benign requests, which has sparked discussions on balancing ethical considerations and practical functionality. This suggests that Claude Instant may have a lower likelihood of producing false statements compared to Claude 3 due to its focus on usability and performance.",
  "gold_answer_marketing": "Claude Instant is a faster, less expensive, and lighter version of Claude with a shorter input context length. Claude 3 has faced criticism for ethical alignment issues that may affect usability and performance."},
24: {"question": "Who developed the language model family known as Chinchilla?",
  "gold_answer_research": "The Chinchilla language model family was developed by the research team at DeepMind and presented in March 2022. It is named 'Chinchilla' as an advancement over the previous Gopher model family. The Chinchilla family has been trained to investigate the scaling laws of large language models and is designed to outperform GPT-3.",
  "gold_answer_marketing": "The research team at DeepMind developed the language model family known as Chinchilla."},
25: {"question": "What benchmark did Chinchilla achieve an average accuracy of 67.5% on?",
  "gold_answer_research": "Chinchilla achieved an average accuracy of 67.5% on the MMLU benchmark (Measuring Massive Multitask Language Understanding).",
  "gold_answer_marketing": "Chinchilla achieved an average accuracy of 67.5% on the MMLU benchmark (Measuring Massive Multitask Language Understanding)."},
27: {"question": "What is the relationship between Chinchilla and the Gopher language model families?",
  "gold_answer_research": "The Chinchilla family of transformer models is essentially the same as the Gopher family, with minor modifications and different training optimizers. Chinchilla uses AdamW optimizer while Gopher uses Adam optimizer. Additionally, Chinchilla uses relative positional encoding and RMSNorm instead of absolute positional encoding and LayerNorm used by Gopher. Chinchilla has 70B parameters and outperforms Gopher on the MMLU benchmark by 7%, showcasing an improvement in performance. Both families follow similar naming conventions and were developed to investigate the scaling laws of large language models.",
  "gold_answer_marketing": "Chinchilla is a family of transformer models developed by DeepMind, which is a further development over a previous model family named Gopher. Both model families were trained to investigate the scaling laws of large language models."},
28: {"question": "What distinguishes the architectures of the Chinchilla and Gopher family models in terms of optimization techniques used?",
  "gold_answer_research": "The main distinction in optimization techniques between the Chinchilla and Gopher family models lies in the choice of optimizers. The Gopher family utilizes the Adam optimizer, whereas the Chinchilla family is trained using the AdamW optimizer. Additionally, the Gopher family employs RMSNorm instead of LayerNorm, and relative positional encoding rather than absolute positional encoding. These differences in optimization techniques contribute to the unique characteristics and performance of each model family.",
  "gold_answer_marketing": "The Chinchilla family uses AdamW optimizer, while the Gopher family uses the Adam optimizer."},
30: {"question": "What is the recommended strategy for training large autoregressive language models with limited compute resources, as contributed by the Chinchilla team?",
  "gold_answer_research": "The Chinchilla team recommends that the number of training tokens should be doubled for every model size doubling to achieve better results on downstream tasks. They also suggest using larger, higher-quality training datasets to improve performance. Additionally, they mention the importance of balancing model size and efficiency to address computational costs and inference latency limitations. It is advised to focus on Transformer language models and consider sharing model parameters for quick task-switching when deploying as a service.",
  "gold_answer_marketing": "The Chinchilla team recommends doubling the number of training tokens for every model size doubling and using larger, higher-quality training datasets to achieve better results on downstream tasks."},
33: {"question": "What are some key areas of research in the field of artificial intelligence as reflected in recent academic literature?",
  "gold_answer_research": "Recent academic literature in the field of artificial intelligence reflects key areas of research such as natural language processing with state-of-the-art transformers, feature learning in infinite-width neural networks, diverse beam search for complex scene description, and the development of generative AI models capable of generating text and images. Additionally, research focuses on human preferences in dueling bandits, the use of few-shot learners in language models, and the exploration of knowledge-grounded neural conversation models. These areas of research highlight the advancements in AI technology and its applications across various domains.",
  "gold_answer_marketing": "Some key areas of research in artificial intelligence include natural language processing, deep neural networks, generative AI, AI safety, AI art, reinforcement learning, and language agents alignment."},
34: {"question": "What are some of the limitations of traditional position encoding methods in the architecture of pre-trained language models (PLMs), and what novel approach does the paper propose to address these issues?",
  "gold_answer_research": "One limitation of traditional position encoding methods in PLMs is that they may not enable length extrapolation of pre-existing models, leading to the need for substantial pre-training costs. The paper proposes a novel approach called Position Interpolation, which extends existing PLMs without deviating far from existing definitions of position encoding or attention mechanisms. This method allows for much extended context windows for text modeling, leading to significant perplexity gains and improved model performance.",
  "gold_answer_marketing": "Traditional position encoding methods in PLMs have limitations in enabling length extrapolation and adapting to extended context windows. The paper proposes a novel approach called Position Interpolation, which generates strong models that can effectively make use of much extended context windows. This method allows for substantial pre-training cost savings and preserves the quality of the original models, even for small context window tasks."},
35: {"question": "How does the Rotary Position Embedding (RoPE) approach in Transformers differ from the traditional additive method of position embedding with respect to encoding position information?",
  "gold_answer_research": "The RoPE approach in Transformers differs from the traditional additive method of position embedding by being multiplicative instead of additive. While traditional methods add position encoding to context representations, RoPE incorporates relative position information through rotation matrix product. This means that RoPE naturally includes relative position dependency in the self-attention formulation, without altering terms in the expanded formulation like the additive method does. Additionally, RoPE's properties show that it decays as the relative distance between positions increases, providing a clear theoretical interpretation of how position information is encoded.",
  "gold_answer_marketing": "The RoPE approach in Transformers differs from the traditional additive method of position embedding by incorporating relative position information through rotation matrix product instead of altering terms in the expanded formulation of additive position encoding."},
36: {"question": "What is the significance of comparing the normalized subspace similarity between ∆Wq, ∆Wv, and random Gaussian matrices when analyzing the adaptation of pre-trained language models?",
  "gold_answer_research": "Comparing the normalized subspace similarity between ∆Wq, ∆Wv, and random Gaussian matrices provides insight into the underlying mechanism for adapting pre-trained language models. It helps determine the intrinsic rank of the adaptation matrix ∆W and sheds light on the connection between ∆W and the original weight matrix W. By analyzing these similarities, we can understand how much of the adaptation is specific to the task at hand and how much is influenced by the pre-trained model. This comparison is crucial for optimizing the adaptation process and maximizing downstream performance in NLP tasks.",
  "gold_answer_marketing": "Comparing the normalized subspace similarity between ∆Wq, ∆Wv, and random Gaussian matrices helps understand the underlying mechanism for adapting pre-trained language models. It reveals the intrinsic rank and common singular value directions learned by different runs, shedding light on the fundamental principles of using pre-trained language models for downstream tasks in NLP."},
38: {"question": "What issues are associated with the homogeneity of language model training contractors, and how might it affect the behavior of the models?",
  "gold_answer_research": "The issues associated with the homogeneity of language model training contractors include potential biases in the labeling process, lack of diverse perspectives leading to limited coverage of sensitive content, and reduced robustness in model performance across different tasks. This homogeneity can affect the behavior of the models by reinforcing certain biases, increasing the risk of harmful content generation, and limiting the models' ability to generalize effectively. To address these issues, it is important to ensure diversity among labelers, incorporate varied perspectives in training data, and implement measures to enhance model robustness and performance across a range of tasks.",
  "gold_answer_marketing": "The homogeneity of language model training contractors can lead to biased or limited perspectives in the data, which may result in the models producing harmful content, gaming objectives, or lacking sensitivity to diverse viewpoints. This can affect the behavior of the models by reinforcing stereotypes, increasing toxicity, and reducing their ability to accurately represent under-represented groups."},
39: {"question": "What are common research topics and themes found in recent publications about artificial intelligence and natural language processing?",
  "gold_answer_research": "Recent publications in artificial intelligence and natural language processing have covered topics such as transformer models, feature learning in neural networks, attention mechanisms, multi-task benchmark platforms, semantic search using sentence embeddings, cross-task generalization, and question generation for question answering. Themes commonly explored include machine comprehension of text, reinforcement learning algorithms, sentence embeddings, semantic compositionality, reasoning with language models and knowledge graphs, and the gap between neural text and human text. These publications also delve into deep language understanding, retrieval-augmented transformers, image captioning, and open datasets for image-text pairs.",
  "gold_answer_marketing": "Common research topics and themes in recent publications on artificial intelligence and natural language processing include transformer models, attention mechanisms, semantic search, sentence embeddings, and question answering using language models and knowledge graphs."},
41: {"question": "Question: When conducting demographic and technical assessments of teams or research subjects, what types of data categories are typically collected and analyzed to ensure a comprehensive understanding of the group's composition and the methods used?",
  "gold_answer_research": "When conducting demographic and technical assessments of teams or research subjects, it is important to collect and analyze data categories such as age, gender, education level, professional background, and expertise in specific areas. By gathering information on these categories, you can ensure a comprehensive understanding of the group's composition and the methods used in your assessments. Additionally, it may be helpful to consider factors like cultural background, language proficiency, and geographical location to capture a more nuanced picture of the group being assessed. This detailed approach to data collection and analysis can provide valuable insights for making informed decisions and recommendations based on the gathered information.",
  "gold_answer_marketing": "Answer: Demographic data such as age, gender, education level, and technical data related to skills and experience are typically collected and analyzed for comprehensive understanding."},
43: {"question": "What kind of tasks can be performed using the datasets described in the provided text, and what are some common features of these datasets?",
  "gold_answer_research": "The datasets described in the provided text can be used for tasks such as question answering, duplicate question retrieval, entity retrieval, citation prediction, query understanding, document understanding, passage retrieval, text summarization, fact verification, and code search. Common features of these datasets include diverse task categories, comprehensive instructions, a wide range of synthetic user personalities and interaction patterns, and a focus on enhancing comprehension of documents to deliver accurate results. Additionally, the datasets cover a variety of domains such as public health, scientific exams, climate, and general knowledge.",
  "gold_answer_marketing": "The datasets described in the provided text can be used for tasks such as question answering, document summarization, duplicate question retrieval, code search, sentence simplification, dialogue generation, body retrieval, caption generation, fact verification, and more. Some common features of these datasets include diverse input-output pairs, incorporation of various knowledge-intensive datasets, and a focus on generating high-quality synthetic data points."},
44: {"question": "What conclusions can be drawn about the relationship between input prompt toxicity and output toxicity when using different language models and prompts?",
  "gold_answer_research": "Based on the findings presented in the results section, it can be concluded that the relationship between input prompt toxicity and output toxicity varies depending on the language model used and the specific prompt given. When instructed to produce a safe and respectful output, InstructGPT models generate less toxic outputs compared to GPT-3, but this advantage disappears when the respectful prompt is removed. On the other hand, when explicitly prompted to produce a toxic output, InstructGPT outputs are much more toxic than GPT-3 outputs. Additionally, the toxicity of the model outputs is highly correlated with the toxicity of the input prompt, as shown in Figure 39.",
  "gold_answer_marketing": "The study found that when instructed to produce a safe and respectful output, InstructGPT models generate less toxic outputs compared to GPT-3. However, this advantage disappears when the respectful prompt is removed. Interestingly, when explicitly prompted to produce a toxic output, InstructGPT outputs are much more toxic than GPT-3. This suggests that the toxicity of the output is highly correlated with the toxicity of the input prompt."},
45: {"question": "What are some challenges in training retrieval systems and how are negative samples used to address them?",
  "gold_answer_research": "Training retrieval systems face challenges such as redundancy in retrieved documents and lack of diversity in retrieval. Negative samples, including randomly sampled negatives, denoised hard negatives, and instruction-unfollowing negatives, are crucial for improving system performance. Carefully designed negative samples help the system effectively learn the task, but they can also lead to performance drops in out-of-domain datasets. Combining random samples and challenging negatives during training is key to building a competitive system for both in-domain and out-of-domain retrieval.",
  "gold_answer_marketing": "Some challenges in training retrieval systems include high cost of annotating datasets for new tasks and improving performance in zero-shot settings. Negative samples, such as denoised hard negative documents and instruction-unfollowing negative documents, are used to train retrieval systems effectively and address performance drops in out-of-domain datasets."},
46: {"question": "What factors have been found to potentially impact the ability of models to follow instructions, based on the analysis provided?",
  "gold_answer_research": "Based on the analysis provided, factors that have been found to potentially impact the ability of models to follow instructions include the human feedback obtained from contractors, which may be influenced by their beliefs, cultural backgrounds, and personal history. Additionally, the model's behavior can be affected by false premises in instructions, tendencies to hedge, and performance degradation with multiple explicit constraints in instructions. The models are also not fully aligned or safe, as they can generate toxic or biased outputs, make up facts, and fail to generate reasonable outputs in some cases.",
  "gold_answer_marketing": "Factors that may impact the ability of models to follow instructions include false premises in instructions, models hedging unnecessarily, performance degradation with multiple constraints in instructions, generation of toxic or biased outputs, and over-generalization leading to refusal of innocuous instructions."},
47: {"question": "What are some key factors to consider when building a successful multi-task instruction-following retrieval system as identified in the research?",
  "gold_answer_research": "Some key factors to consider when building a successful multi-task instruction-following retrieval system include the need for cross-task interdependence for training a single retriever, the flexibility and zero-shot transfer enabled by instructions compared to task identifiers, and the elimination of the need for hosting multiple task-specific retrievers. Additionally, optimizing the mix and volume of instructional data for diverse tasks is crucial, as well as considering the impact of ranking strategy in data construction. Finally, the effectiveness of the dataset scale in retrieval and the importance of carefully designed negative samples should be taken into account for improved efficiency of instruction-following retrievers.",
  "gold_answer_marketing": "Key factors to consider when building a successful multi-task instruction-following retrieval system include the effectiveness of the dataset scale in retrieval, the diversity in data and model scale, carefully designed negative samples, and the ability to adapt to new tasks via instructions."},
48: {"question": "What are the benefits of using retrieval-augmented techniques in multimodal language modeling, as demonstrated by the performance of the RA-CM3 model in the document?",
  "gold_answer_research": "The benefits of using retrieval-augmented techniques in multimodal language modeling, as demonstrated by the performance of the RA-CM3 model, include significantly better training efficiency with less training compute, outperforming existing models by using less training data, compute, and parameters. The retrieval augmentation allows the model to focus on learning how to use retrieved documents in context, leading to improved accuracy in classification tasks. Additionally, the RA-CM3 model achieves strong performance in image and caption generation, surpassing existing models like DALL-E and Flamingo despite using fewer resources.",
  "gold_answer_marketing": "The benefits of using retrieval-augmented techniques in multimodal language modeling, as demonstrated by the performance of the RA-CM3 model in the document, include outperforming existing models by using less training data, compute, and parameters, achieving significantly better training efficiency, and improving accuracy in k-shot classification tasks. Additionally, retrieval augmentation allows the model to focus on learning how to use retrieved documents in context, leading to stronger performance in tasks such as image and caption generation."},
50: {"question": "What methods are typically employed to create training data for embedding models that use task-specific instructions?",
  "gold_answer_research": "To create training data for embedding models that use task-specific instructions, a common method is to combine datasets from different sources, such as the SuperNaturalInstructions dataset with existing collections designed for embedding training. The SuperNaturalInstructions dataset provides natural language instructions, which can be paired with positive and negative examples to form training samples. Additionally, for tasks like classification or similarity, training samples can be constructed by selecting text sequences associated with different classes or similarities. This diverse training data is essential for instruction-based finetuning, which enables the embedding model to learn from a wide range of tasks and domains.",
  "gold_answer_marketing": "Training data for embedding models that use task-specific instructions is typically created by formulating a wide variety of tasks as text-to-text problems, distinguishing good/bad candidate outputs given an input text. This is done by combining datasets with natural language instructions and constructing positive and negative pairs for training."},
51: {"question": "Question: What are some of the challenges and innovations associated with fine-tuning large language models, and how does the approach discussed in the referenced text aim to address them?",
  "gold_answer_research": "Some challenges associated with fine-tuning large language models include limited access to and manipulation of knowledge, lagging performance on knowledge-intensive tasks, and the need for provenance in decision-making and updating world knowledge. The approach discussed in the referenced text aims to address these challenges by utilizing Retrieval Augmented Generation (RAG), which involves retrieving relevant passages from a corpus to feed to the language model for improved performance in tasks such as question-answering and dialogue. This iterative approach focuses on improving alignment with user intent and fine-tuning models to control sentiment and improve response quality in various language tasks.",
  "gold_answer_marketing": "The challenges with fine-tuning large language models include aligning them with user intent and controlling the quality of generated outputs. The approach discussed in the referenced text aims to address these challenges by using Retrieval Augmented Generation (RAG) to retrieve relevant passages from a corpus and feed them to the language model, improving alignment and performance."},
52: {"question": "What is a common technique used to address the outlier issue when applying block-wise k-bit quantization to input tensors, and how does it work?",
  "gold_answer_research": "A common technique used to address the outlier issue when applying block-wise k-bit quantization to input tensors is to chunk the input tensor into blocks that are independently quantized, each with their own quantization constant. This approach involves dividing the input tensor into contiguous blocks of size B by flattening the tensor and slicing it into n blocks, where n is determined by the size of the blocks. Each block is then quantized independently using a quantization constant c, which helps prevent outlier values from causing performance degradation.",
  "gold_answer_marketing": "A common technique used to address the outlier issue when applying block-wise k-bit quantization to input tensors is to chunk the input tensor into blocks that are independently quantized, each with their own quantization constant. This helps prevent performance degradation by reducing the impact of outliers on the quantization process."},
54: {"question": "What considerations or techniques are commonly implemented when setting up finetuning experiments for machine learning models?",
  "gold_answer_research": "When setting up finetuning experiments for machine learning models, it is common to use a two-stage approach. The initial stage involves setting the initial parameters using a language modeling objective. This is followed by a supervised discriminative 'fine-tuning' stage to adapt these parameters to the target task. Additionally, it is typical to train all models using the Adam optimizer and a triangular learning rate scheduler with 10% warmup. Experimentation with different hyperparameters such as number of epochs, peak learning rate, and batch size is also conducted to optimize model performance. Finally, utilizing a mixture of datasets and balancing the sizes of datasets can help improve the robustness and generalization of the finetuned models.",
  "gold_answer_marketing": "Considerations for setting up finetuning experiments for machine learning models commonly include using a language modeling objective for initial parameter setting and supervised discriminative fine-tuning for adapting parameters to the target task. Techniques such as hyperparameter search, Adam optimizer with triangular learning rate scheduler, and balancing dataset sizes through mixing strategies are also commonly implemented. Additionally, freezing some model layers during fine-tuning and incorporating negative examples for contrastive learning can be effective strategies."},
55: {"question": "What are the implications of the equivalence relation defined in the theoretical analysis of the DPO model for understanding the relationship between reward functions in reinforcement learning?",
  "gold_answer_research": "The equivalence relation defined in the theoretical analysis of the DPO model implies that two reward functions are considered equivalent if they differ by a constant function. This means that the class of learned reward models is not constrained by this reparameterization, allowing for the exact recovery of the optimal policy. Understanding this relationship between reward functions in reinforcement learning helps in defining a unique reward function within each equivalence class, which is crucial for optimizing policies under existing models of human preferences. It also highlights the generality and flexibility in the reward model due to the proposed reparameterization.",
  "gold_answer_marketing": "The equivalence relation defined in the theoretical analysis of the DPO model shows that two reward functions are considered equivalent if they differ by a fixed function. This implies that different reward functions can lead to the same optimal policy, allowing for flexibility in designing reward models in reinforcement learning."},
59: {"question": "Considering the structure and content of the provided text, what guidelines should be used to evaluate the effectiveness of a summary or chatbot response in this context?",
  "gold_answer_research": "To evaluate the effectiveness of a summary or chatbot response in this context, guidelines should include assessing the faithfulness of the answer to the retrieved context, the relevance of the answer to the question, and the focus of the retrieved context. Additionally, consider using quality metrics such as answer relevancy to rank responses based on how directly they address the question and avoid redundant or incomplete information. Lastly, take into account the performance of different tasks such as summarization, citation prediction, and passage ranking to determine the overall effectiveness of the response.",
  "gold_answer_marketing": "Answer: Evaluate based on faithfulness, answer relevance, and context relevance."},
60: {"question": "What are some recent methods and technologies that have been developed to enhance the capabilities and performance of natural language processing models?",
  "gold_answer_research": "Recent methods and technologies developed to enhance natural language processing models include retrieval-augmented multimodal language modeling, which outperforms existing models with less training data and parameters. Another advancement is the use of feature learning in infinite-width neural networks to improve performance. Additionally, embedding techniques in NLP have been developed to map words or phrases to real number vectors, enhancing the model's understanding of language. These innovations have led to improvements in tasks like query reformulation, document ranking, and fine-tuning larger language models for various applications.",
  "gold_answer_marketing": "Recent methods and technologies include retrieval-augmented language models, feature learning in infinite-width neural networks, and word embeddings."},
61: {"question": "What are some potential directions for future work mentioned in the document related to enhancing question-answering techniques for document-oriented tasks?",
  "gold_answer_research": "One potential direction for future work mentioned in the document is the development of multi-modal approaches that incorporate table and figure information into GPT-4 question-answering for documents. Another direction is to incorporate question type in the PDFTriage approach to improve the efficiency and efficacy of the approach. Additionally, the document suggests further research in document-grounded, information-seeking question answering, which the dataset is designed to facilitate.",
  "gold_answer_marketing": "Some potential future directions mentioned in the document include developing multi-modal approaches that incorporate table and figure information into question-answering for documents, and incorporating question type in the PDFTriage approach to improve efficiency and efficacy."},
62: {"question": "What information would you expect to find in section 2 of a document, based on the types of questions classified under Summarization?",
  "gold_answer_research": "Based on the types of questions classified under Summarization, you would expect to find key takeaways, concise summaries, and specific content extraction related to different sections of the document in section 2. The section likely contains detailed summaries of specific parts of the document, along with structured metadata representation and instructions for summarizing the content effectively. It may also include guidelines for extracting specific information and rewriting text for clarity and conciseness.",
  "gold_answer_marketing": "Based on the types of questions classified under Summarization, you would expect to find key takeaways, concise summaries, and specific content extraction related to the document in section 2."},
63: {"question": "What are the main advantages and attention mechanisms that contribute to the enhanced performance and efficiency of the newly introduced language model as compared to its predecessors?",
  "gold_answer_research": "The main advantages of the newly introduced language model include utilizing retrieval-augmentation to incorporate external knowledge, which improves prediction accuracy. Additionally, the model employs attention mechanisms that allow for better understanding of dependencies between source and target sequences, leading to more informed predictions. These attention mechanisms have been extended from machine translation to various other fields, enhancing the model's adaptability and performance across different tasks. Finally, the model's use of self-attention mechanisms enables better contextual representation learning, parallelization, and modeling of longer intra-token relations, improving efficiency and performance compared to previous models.",
  "gold_answer_marketing": "The main advantages of the newly introduced language model include the use of retrieval-augmented mechanisms, attention mechanisms, and context representation learning, which contribute to enhanced performance and efficiency compared to its predecessors."},
64: {"question": "What criteria are used to assess the quality of recommendations provided by different language models in a comparison study?",
  "gold_answer_research": "In a comparison study of language models, criteria such as sentence relevance, lexical accuracy, and contextual understanding are used to assess the quality of recommendations. Different tasks may benefit from different evaluation measures, such as STRINC, LEXICAL, and CXMI. Additionally, template selection plays a vital role in the quality of recommendations, with deliberate template design being important for tasks like query suggestion. The overall quality of recommendations is often judged using a Likert scale, along with metadata collection for each model output.",
  "gold_answer_marketing": "The criteria used to assess the quality of recommendations provided by different language models in a comparison study include comparing to human-created benchmarks, examining intrinsic character, comparing two models, investigating rate of learning, and analyzing learning curves."},
65: {"question": "What approaches have been proposed to enhance the task performance of language models while considering the trade-offs such as runtime efficiency, robustness to irrelevant context, and attribution quality?",
  "gold_answer_research": "Several approaches have been proposed to enhance the task performance of language models while considering trade-offs. These include using compression and selective augmentation methods to decrease the propensity of models to generate toxic or biased outputs. Adversarial setups have been suggested where labelers find worst-case behaviors of the model and add them to the dataset. Additionally, models like BART and T5 leverage bi-directional attention to achieve stronger performance on both discriminative and generative tasks. These methods aim to balance model performance with considerations such as runtime efficiency, robustness to irrelevant context, and attribution quality.",
  "gold_answer_marketing": "Approaches proposed to enhance language model task performance include compression and selective augmentation, adversarial set-ups for labeling worst-case behaviors, retrieval-augmented models, and extending existing models to enable length extrapolation while maintaining quality."},
67: {"question": "What metrics are commonly used to compare the performance of language models in various tasks, as outlined in an experimental results table?",
  "gold_answer_research": "Common metrics used to compare the performance of language models in various tasks, as outlined in an experimental results table, include Exact Match and Unigram F1. These metrics have become standard in evaluating language models. Additionally, other metrics such as BLEU score, FactScore (factuality), precision, and recall are also commonly used to assess the performance of language models across different tasks. It is important to consider a variety of metrics to get a comprehensive understanding of the effectiveness of a language model in different contexts.",
  "gold_answer_marketing": "The metrics commonly used to compare the performance of language models in various tasks are Exact Match and Unigram F1."},
69: {"question": "What is the role of manual assessment in the validation of language model predictions according to the text provided?",
  "gold_answer_research": "Manual assessment plays a crucial role in the validation of language model predictions. The engineers evaluate the quality of model outputs by having labelers rate them on test sets consisting of prompts from held-out customers. This manual assessment helps ensure that the models are aligned with a broad distribution of language tasks and can identify any behavioral issues that may arise from misalignment. Additionally, human annotators find that certain reflection token predictions are aligned with their assessments, providing valuable insights into the accuracy and effectiveness of the models.",
  "gold_answer_marketing": "Answer: Manual assessment plays a key role in evaluating the quality of language model predictions by having labelers rate the model outputs and comparing them to prompts from held-out customers."},
70: {"question": "What are the general steps outlined for training a language model in the document, and how is the training data for the generator language model collected and utilized?",
  "gold_answer_research": "The document outlines the general steps for training a language model, including incorporating retrieved documents into the main input sequence and optimizing the loss function to train the generator. The training data for the generator language model is collected through various techniques such as supervised fine-tuning, critic learning, and custom retrievers for downstream tasks. The collected data is used to train the generator on specific tasks like summarization, machine reading comprehension, and natural language to SQL translation, improving performance on those tasks.",
  "gold_answer_marketing": "The general steps for training a language model include fine-tuning on specific datasets, filtering pretraining data, and using critic learning. Training data for the generator language model is collected from open-access NLP papers and used for downstream conditional text generation tasks."},
73: {"question": "What are the three main categories used to refine language model abilities in understanding and executing search tasks according to the given document?",
  "gold_answer_research": "The three main categories used to refine language model abilities in understanding and executing search tasks are query understanding, document understanding, and query-document relationship understanding. Tasks within these categories focus on interpreting queries, comprehending documents, and understanding the relationships between queries and documents. This approach aims to enhance the models' performance in interpreting and responding to search-related instructions effectively, improving their utility in complex information retrieval scenarios.",
  "gold_answer_marketing": "The three main categories used to refine language model abilities in understanding and executing search tasks are query understanding, document understanding, and query-document relationship understanding."},
74: {"question": "What are some of the emerging research topics and challenges in the field of natural language processing and information retrieval according to recent academic conferences and publications?",
  "gold_answer_research": "Recent academic conferences and publications have highlighted emerging research topics and challenges in natural language processing and information retrieval. Some key areas of focus include efficient retrieval augmented generation, unsupervised dense information retrieval with contrastive learning, citation-informed transformers, and knowledge refinement via interaction between search engines and large language models. Additionally, challenges such as zero-shot retrieval, semantic search using GPT sentence embeddings, and prompt-based effective input reformulation for legal case retrieval have been identified as important research directions. These topics reflect the ongoing advancements and complexities in the field, driving innovation and progress in NLP and IR research.",
  "gold_answer_marketing": "Some emerging research topics and challenges in the field of natural language processing and information retrieval include efficient generation from unstructured knowledge, semantic code search evaluation, unsupervised dense information retrieval, context-aware document term weighting, knowledge refinement through interaction with large language models, and investigating the effectiveness of large language models in search re-ranking."},
75: {"question": "Question: How do models with different fine-tuning strategies compare in terms of accuracy and F1 score for fact verification tasks?",
  "gold_answer_research": "Models with different fine-tuning strategies are compared in terms of accuracy and F1 score for fact verification tasks. The introduction of LLMs has led to notable developments, with some studies leveraging prompting methods to apply LLMs in IR tasks. However, not all LLMs consistently outperform fine-tuned smaller models. For example, RankGPT based on gpt-3.5-turbo underperforms monoBERT in certain scenarios. Fine-tuning is not strictly necessary for models like GPT3, which has been evaluated on closed book question answering tasks without any updates or fine-tuning.",
  "gold_answer_marketing": "Models with different fine-tuning strategies have shown mixed results in terms of accuracy and F1 score for fact verification tasks. Some studies have found that large language models (LLMs) outperform smaller fine-tuned models, while others have reported inconsistent performance. Factors such as task complexity and the need for prompt methods to apply LLMs in information retrieval tasks can also impact the comparison."},
76: {"question": "What components does a fact verification task typically involve in order to assess the accuracy of a given statement?",
  "gold_answer_research": "A fact verification task typically involves assessing the relationship between a claim and the evidence provided, analyzing if there is enough information for a conclusive judgment. This task requires a detailed understanding of the claim and evidence to determine if it is supported or refuted. The use of performance metrics based on including gold answers in model generations instead of exact matching can help search engines deliver accurate and relevant results. Additionally, incorporating lexical measures and verification functions can aid in determining the accuracy of statements.",
  "gold_answer_marketing": "A fact verification task typically involves assessing the relationship between a claim and supporting evidence to determine accuracy."},
78: {"question": "What are the key factors that determine the performance of HALO-aligned models compared to non-HALO models, according to the results presented in the analysis?",
  "gold_answer_research": "According to the analysis presented, the key factors that determine the performance of HALO-aligned models compared to non-HALO models include the specific alignment method used (such as DPO and PPO variant), the model size (significant gap at 13B+ model sizes), and the ability to match or exceed the generation quality of SFT target sequences. Additionally, the study suggests that the cost of increasing model alignment is modest relative to pretraining, and that the modeling of human biases in HALOs may have practical benefits in improving overall performance.",
  "gold_answer_marketing": "The key factor that determines the performance of HALO-aligned models compared to non-HALO models is the model size, with HALO-aligned models generally outperforming non-HALO models at larger sizes (13B+ model sizes)."},
80: {"question": "How does the performance of KTO compare to DPO in model alignment, and what are the potential implications for data usage and training efficiency?",
  "gold_answer_research": "Based on the provided data and experiments, KTO consistently outperforms DPO in model alignment, even with restrictions such as using only one output per input. This suggests that KTO can achieve higher win rates and improve performance across various benchmarks compared to DPO. The implications of this performance difference include the ability to achieve quality generation results with significantly fewer desirable examples, potentially leading to more efficient data usage and training processes. This indicates that KTO may offer a more efficient and effective approach to model alignment compared to DPO.",
  "gold_answer_marketing": "KTO outperforms DPO in model alignment with up to 90% fewer examples. This suggests that KTO can achieve high performance even with imbalanced data, potentially leading to more efficient training processes."},
81: {"question": "What are some common approaches to building an open-domain question answering system?",
  "gold_answer_research": "Some common approaches to building an open-domain question answering system include using the RAG model, which minimizes the negative log-likelihood of answers, and comparing it to extractive QA paradigms that rely on non-parametric knowledge retrieval. Another approach is to incorporate question rewriting techniques to make open-domain QA more conversational. Additionally, utilizing datasets like QASPER, which contain questions requiring complex reasoning, can improve the performance of the system. References to papers by Anantha et al. and Asai et al. provide further insights into building ODQA systems.",
  "gold_answer_marketing": "Common approaches to building an open-domain question answering system include using retrieval over a knowledge base and incorporating the retrieved content as part of the prompt. Other methods involve pretraining models on large amounts of text data and fine-tuning them for question answering tasks."},
82: {"question": "What is the difference between open-book and closed-book question answering?",
  "gold_answer_research": "Open-book question answering involves the use of external sources of knowledge, such as Wikipedia, to retrieve information and generate a response. In contrast, closed-book question answering relies on pre-trained language models that have memorized factual knowledge within their parameters to generate responses without explicit context. Closed-book QA can be seen as analogous to a closed-book exam where no external resources are allowed. The key distinction lies in the reliance on external knowledge sources for open-book QA versus internal memorized knowledge for closed-book QA.",
  "gold_answer_marketing": "Open-book question answering involves using external sources of knowledge to answer questions, while closed-book question answering relies on pre-trained language models to provide answers without explicit context."},
84: {"question": "What are the basic components of the Retriever-Reader framework in open-domain QA?",
  "gold_answer_research": "The basic components of the Retriever-Reader framework in open-domain QA include a retriever model, which fetches relevant information based on input prompts efficiently using FAISS. The retriever component is responsible for retrieving contextually relevant documents or evidence blocks based on the input question. The reader component then processes this retrieved information to generate answers to the questions posed. This framework combines information retrieval and machine reading comprehension to achieve state-of-the-art results in open-domain question answering tasks.",
  "gold_answer_marketing": "The basic components of the Retriever-Reader framework in open-domain QA are the retriever and the reader components, which can be set up and trained independently or jointly trained end-to-end. The retriever component automatically fetches relevant information based on input prompts, while the reader component processes and comprehends the retrieved information to answer questions."},
85: {"question": "How is the TF-IDF model used in question answering retrieval systems?",
  "gold_answer_research": "In question answering retrieval systems, the TF-IDF model is used to represent queries and documents as bag-of-word vectors with terms weighted by term frequency multiplied by inverse document frequency. This allows for efficient non-learning-based search engine operations based on the vector space model. The TF-IDF model helps in calculating the relevance of documents to queries by measuring the importance of terms in the context of the entire document collection. This classic information retrieval approach aids in retrieving relevant information to answer questions accurately and efficiently.",
  "gold_answer_marketing": "The TF-IDF model is used in question answering retrieval systems to weight terms in queries and documents based on their importance in determining relevance."},
86: {"question": "Can neural networks enhance the process of information retrieval in QA systems?",
  "gold_answer_research": "Neural networks, such as MLP, LSTM, and bidirectional LSTM, can be used to learn dense representations of text for information retrieval in QA systems. These approaches, known as 'Neural IR', are a new category of methods that can improve performance in retrieval problems. The introduction of neural retrievers in recent QA literature has shown to outperform traditional word-similarity-based architectures, such as BM25, and can scale to handle knowledge-grounded dialogue tasks effectively. Additionally, incorporating pre-trained retrievers in QA systems has been shown to enhance the performance of generative language models.",
  "gold_answer_marketing": "Yes, neural networks can enhance the process of information retrieval in QA systems by improving performance in open-domain QA tasks and enabling the generation of more accurate answers."},
87: {"question": "What is the importance of fine-tuning in the context of QA data for open-domain question answering models?",
  "gold_answer_research": "Fine-tuning is important in the context of QA data for open-domain question answering models because it allows the model to adapt and improve its performance on specific QA datasets. By fine-tuning the model with common QA datasets, engineers can optimize the model's ability to answer questions accurately. However, there is a concern about the significant overlap between questions in the train and test sets of public QA datasets, which could affect the generalization ability of the fine-tuned models. Engineers should carefully consider this overlap and potentially explore ways to mitigate its impact during the fine-tuning process to ensure the model's effectiveness in real-world applications.",
  "gold_answer_marketing": "Fine-tuning is important in the context of QA data for open-domain question answering models to improve search task performance and the ability to generalize to unseen datasets."},
88: {"question": "How does pre-training with tasks like the Inverse Cloze Task benefit open-domain question answering models?",
  "gold_answer_research": "Pre-training with tasks like the Inverse Cloze Task benefits open-domain question answering models by improving the retrieval process over a knowledge base. By predicting the context given a sentence, the model can better understand the relationship between the question and the evidence. This approach helps in incorporating retrieved content effectively into the prompt, leading to higher accuracy in the question answering task. Additionally, using models pretrained with ICT can enhance the overall performance of the QA system by providing a better understanding of the context.",
  "gold_answer_marketing": "Pre-training with tasks like the Inverse Cloze Task benefits open-domain question answering models by improving retrieval and generation steps, ultimately enhancing the accuracy of the process."},
89: {"question": "What is the main goal of prompt engineering in language models?",
  "gold_answer_research": "The main goal of prompt engineering in language models is to effectively steer the behavior of the model towards desired outcomes without updating the model weights. This is achieved by composing and formatting prompts in a way that maximizes the model's performance on a specific task. Prompt engineering involves treating prompts as trainable parameters and optimizing them directly on the embedding space through methods like AutoPrompt, Prefix-Tuning, P-tuning, and Prompt-Tuning. The ultimate aim is to enhance the model's performance and alignment with user-defined tasks.",
  "gold_answer_marketing": "The main goal of prompt engineering in language models is to steer the behavior of the model for desired outcomes without updating the model weights."},
91: {"question": "What are some known biases that can affect the performance of few-shot classification in LLMs?",
  "gold_answer_research": "Some known biases that can affect the performance of few-shot classification in LLMs include majority label bias, recency bias, and common token bias. Majority label bias occurs when the distribution of labels among examples is unbalanced, recency bias refers to the tendency for the model to repeat the label at the end, and common token bias indicates that LLM tends to produce common tokens more often than rare tokens. These biases can contribute to high variance in few-shot classification tasks and may impact the model's ability to generalize effectively.",
  "gold_answer_marketing": "Some known biases that can affect the performance of few-shot classification in LLMs are majority label bias, recency bias, and common token bias."},
92: {"question": "Why might increasing model size not reduce variance in model performance with varying prompts?",
  "gold_answer_research": "Increasing model size may not necessarily reduce variance in model performance with varying prompts because the model's ability to generalize and adapt to different prompts is not solely dependent on its size. Factors such as the quality and relevance of the training examples, the learning rate or schedule, and the model's sensitivity to different hyperparameters can also play a significant role in determining performance variability. Additionally, the complexity of the task or dataset being used for training can impact how effectively the model scales with size. It is essential to consider these factors holistically when optimizing model performance rather than relying solely on increasing model size.",
  "gold_answer_marketing": "Increasing model size may not reduce variance in model performance with varying prompts because the same order of prompts may work well for one model but poorly for another. Additionally, when the validation set is limited, choosing the order of prompts that prevents the model from producing extremely unbalanced predictions or being overconfident can also affect performance."},
93: {"question": "What is the benefit of instruction-based finetuning in language models?",
  "gold_answer_research": "Instruction-based finetuning improves models' ability to generalize to unseen domains and tasks by providing task-specific representations that can be used for many downstream language tasks without additional training. This method also allows pretrained language models to follow instructions provided in prompts, enabling them to generate the desired output given specific inputs. Additionally, instruction finetuning helps transform raw pretrained LLMs into chatbot-like models, making finetuning more accessible and common, particularly for researchers with limited resources. Overall, the benefit of instruction-based finetuning is improved model performance, enhanced generalizability, and reduced communication costs in aligning with human intentions.",
  "gold_answer_marketing": "The benefit of instruction-based finetuning in language models is improved ability to generalize to unseen domains and tasks, without the need for additional training."},
94: {"question": "Can you describe a situation where retrieval-based methods would be necessary to enhance language model performance?",
  "gold_answer_research": "Retrieval-based methods are necessary to enhance language model performance in scenarios where the model needs to generate accurate and informative responses for entity-rich queries, such as 'George Washington standing in front of the Eiffel Tower.' In such cases, incorporating a retrieval module can provide additional context and relevant information to improve the model's understanding and generation of the desired output. Additionally, retrieval-based methods are crucial for question answering tasks, where the model needs to access external knowledge sources to provide accurate and comprehensive answers. By utilizing retrieval mechanisms, the language model can benefit from a wider range of information and improve its performance in handling complex and ambiguous queries effectively.",
  "gold_answer_marketing": "Retrieval-based methods are necessary to enhance language model performance in tasks like question answering, where incorporating additional information from external sources can improve the model's ability to generate accurate and relevant responses."},
95: {"question": "What is the Chain-of-Thought prompting technique and for which types of tasks is it particularly beneficial?",
  "gold_answer_research": "Chain-of-Thought (CoT) prompting is a technique that generates reasoning chains or rationales step by step to lead to a final answer, benefiting complicated reasoning tasks using large models with more than 50B parameters. It can be implemented through iterative Monte Carlo search methods or through a three-step process called augment-prune-select. CoT is particularly beneficial for enhancing model performance on complex tasks by decomposing them into smaller and simpler steps, shedding light on the model's thinking process. Task decomposition in CoT can be done with simple prompting, task-specific instructions, or human inputs.",
  "gold_answer_marketing": "Chain-of-Thought (CoT) prompting is a technique that generates reasoning chains or rationales step by step to lead to a final answer. It is particularly beneficial for complicated reasoning tasks when using large models with more than 50B parameters. Simple tasks only benefit slightly from CoT prompting."},
96: {"question": "How do augmented language models with external tools differ from regular models in functionality?",
  "gold_answer_research": "Augmented language models with external tools, such as TALM and Toolformer, are fine-tuned to learn how to use external tool APIs, expanding their capabilities beyond traditional language processing tasks. These models are trained to incorporate external tool API calls in order to improve the quality of their outputs, allowing them to perform tasks like speech recognition, machine translation, and information retrieval more effectively. By leveraging external tools, these models have the ability to access and utilize a wider range of resources and functionalities, enhancing their overall performance and versatility compared to regular language models.",
  "gold_answer_marketing": "Augmented language models with external tools differ from regular models by fine-tuning a LM to use external tool APIs, expanding the dataset to improve model outputs and enhancing tasks like speech recognition, machine translation, and natural language generation."},
97: {"question": "What can be inferred about the utilization of attention in neural networks?",
  "gold_answer_research": "Attention mechanisms in neural networks play a crucial role in allowing models to focus on specific parts of input data when making predictions or generating outputs. By assigning importance weights to different elements, such as pixels in an image or words in a sentence, attention helps the model to attend to relevant information and make more accurate predictions. The use of attention can improve the interpretability of neural networks by showing which parts of the input data are being focused on during the prediction process. Additionally, attention mechanisms, like multi-head attention, can enhance model performance by allowing the model to jointly attend to information from different representation subspaces at different positions.",
  "gold_answer_marketing": "Attention in neural networks allows the model to focus on specific parts of input data, such as images or text, in order to make predictions or generate output. It helps the model to learn relationships and correlations between different elements and improve performance in tasks like image captioning or language translation."},
101: {"question": "Can the use of attention mechanisms in deep learning models be applied to both machine translation and computer vision?",
  "gold_answer_research": "Yes, attention mechanisms in deep learning models have shown success in both machine translation and computer vision tasks. In machine translation, attention allows the model to capture dependencies between source and target sequences regardless of distance, leading to improved translation quality. Similarly, in computer vision, attention mechanisms have been used to focus on relevant parts of an image during caption generation, showcasing the ability to handle details and global dependencies effectively. Therefore, utilizing attention in both domains can enhance the performance of deep learning models significantly.",
  "gold_answer_marketing": "Yes, attention mechanisms in deep learning models can be applied to both machine translation and computer vision."},
102: {"question": "What are the potential benefits of incorporating self-attention mechanisms into Generative Adversarial Networks (GANs)?",
  "gold_answer_research": "Incorporating self-attention mechanisms into GANs can help the generator and discriminator better model relationships between spatial regions, leading to improved generation of detailed and realistic images. This is particularly useful for capturing global dependencies and enhancing the performance of transformer architectures. Additionally, self-attention can enable the model to assess its own predictions after each generated segment, allowing for customizable decoding algorithms to meet specific constraints or user preferences. Overall, self-attention in GANs can enhance detail handling and overall performance.",
  "gold_answer_marketing": "Incorporating self-attention mechanisms into GANs can help the generator and discriminator better model relationships between spatial regions, leading to improved performance in handling details and capturing global dependencies."},
103: {"question": "How does the transformer model variate from traditional sequence-aligned recurrent architectures?",
  "gold_answer_research": "The transformer model differs from traditional sequence-aligned recurrent architectures by not having a recurrent or convolutional structure. Instead, it heavily relies on self-attention mechanisms for processing sequences. This lack of recurrence and convolution, even with positional encoding, weakly incorporates sequential order, which can be a drawback for tasks sensitive to positional dependencies. Additionally, the transformer's architecture includes embedding layers, sinusoid-wave-based positional encoding, and softmax and linear layers in the final decoder output to maintain position information and facilitate processing of long sequences efficiently.",
  "gold_answer_marketing": "The transformer model differs from traditional sequence-aligned recurrent architectures by not having a recurrent or convolutional structure, and instead making heavy use of self-attention. This allows for handling very long sequences efficiently and achieving better performance on tasks involving long texts."},
104: {"question": "What implications does the concept of a Neural Turing Machine have for the theoretical power of neural networks?",
  "gold_answer_research": "The concept of a Neural Turing Machine (NTM) expands the theoretical power of neural networks by incorporating external memory storage, allowing for more complex computations and tasks. This mimics the Turing machine tape, enabling the neural network to control operation heads for reading and writing to the tape. However, the finite memory in NTM suggests it may resemble more of a 'Neural von Neumann Machine,' limiting its mathematical limitlessness seen in traditional Turing machines. Overall, the addition of external memory in NTM enhances the capabilities and potential applications of neural networks in solving more advanced problems.",
  "gold_answer_marketing": "The concept of a Neural Turing Machine suggests that neural networks can be equipped with external memory storage for more complex operations, potentially increasing their theoretical power."},
}


test_questions = {
4: {"question": "When was the transformer architecture introduced, and by which organization?"},
5: {"question": "How has the accessibility of powerful language models, such as GPT-3 and GPT-4, been controlled by their developers?"},
6: {"question": "What benchmarks or ratings are used to compare the capabilities of different language models?"},
10: {"question": "What are some of the primary applications for language models in technology and computing?"},
14: {"question": "How are language models typically evaluated and what benchmarks are used for this purpose?"},
15: {"question": "What datasets are available for evaluating language processing systems?"},
21: {"question": "What collaborations with other companies have contributed to the development of Claude's capabilities?"},
26: {"question": "According to DeepMind, how should the number of training tokens change relative to the model size?"},
29: {"question": "How do the sizes of models in the Gopher family range?"},
31: {"question": "What type of model architecture do the Gopher and Chinchilla families belong to?"},
32: {"question": "Can you name the author who wrote the novels A Farewell to Arms and The Sun Also Rises?"},
37: {"question": "What are the key advantages of InstructGPT models over GPT-3 models according to the findings in the research?"},
40: {"question": "What metrics are used to compare the performance of different models on training and validation splits according to the document provided?"},
42: {"question": "What types of evaluation metrics are commonly used to assess the accuracy of answers in AI-driven question and answer datasets?"},
49: {"question": "What factors contribute to the performance improvement in retrieval-augmented language models compared to non-retrieval-augmented models?"},
56: {"question": "What are the benchmarks used to evaluate the performance of the Deep Policy Optimization (DPO) method compared to other preference learning algorithms in the document provided?"},
57: {"question": "What methodologies have been evaluated for training language models to align with human preferences, and how do they compare in terms of effectiveness?"},
58: {"question": "What methods have been discussed in the literature for improving the alignment of language models with human preferences or feedback?"},
66: {"question": "What are some of the evaluation metrics used for assessing different types of text generation tasks presented in the study?"},
68: {"question": "Consider a document related to research in natural language processing or artificial intelligence. Can you name some of the recent topics or methods that have been discussed or introduced in the field according to the document?"},
71: {"question": "What is the significance of using reflection tokens in a model like SELF-RAG?"},
72: {"question": "How does the inclusion of selected context as opposed to appending all retrieved text spans impact computational cost during both training and inference times in language model generation tasks?"},
77: {"question": "What are the benefits of modeling human biases in Human-Aware Loss Optimizations (HALOs), and how do they compare to non-HALOs on the same datasets?"},
79: {"question": "What are the modifications made to the traditional Kahneman-Tversky model to adapt it for optimizing language model performance?"},
83: {"question": "How does a model's ability to answer questions relate to its exposure to specific types of questions during training?"},
90: {"question": "How can adding examples to a prompt affect the performance of language models?"},
98: {"question": "What are the main components of a Neural Turing Machine (NTM) architecture?"},
99: {"question": "How might a seq2seq model's limitations be addressed in natural language processing tasks?"},
100: {"question": "What differentiates hard attention from soft attention in image processing algorithms?"},
}


In [ ]:
eng_ans = [inner_dict['gold_answer_research'] for inner_dict in validation_questions_answers.values() if 'gold_answer_research' in inner_dict]

In [ ]:
mkt_ans = [inner_dict['gold_answer_marketing'] for inner_dict in validation_questions_answers.values() if 'gold_answer_marketing' in inner_dict]

In [ ]:
np.max(pd.Series(eng_ans).apply(lambda x: len(x.split('.'))))

7

In [ ]:
np.mean(pd.Series(mkt_ans).apply(lambda x: len(x.split('.'))))

2.493333333333333

### 3.3 Running the RAG System

Let's have a quick look at the validation and test data:

In [ ]:
validation_questions_answers[0]

{'question': 'What purpose do large language models serve in the field of natural language processing?',
 'gold_answer_research': 'Large language models (LLMs) serve the purpose of enabling general-purpose language generation and other natural language processing tasks such as classification. They achieve this by learning statistical relationships from text documents during computationally intensive self-supervised and semi-supervised training. LLMs can be used for text generation by predicting the next token or word, making them valuable for tasks like speech recognition, machine translation, and information retrieval. Additionally, LLMs have superseded previous models like recurrent neural networks, showcasing their efficiency and effectiveness in NLP tasks.',
 'gold_answer_marketing': 'Large language models serve the purpose of improving performance in various natural language processing tasks, such as speech recognition, machine translation, natural language generation, optical cha

In [ ]:
test_questions[4]

{'question': 'When was the transformer architecture introduced, and by which organization?'}

Let's now use the data to ask questions against it. So we need to define our prompt templates, the RAG Chain, etc.

We have two types of User Personas we need to support:

1. The engineers, who require pretty detailed information when they ask questions  
2. The marketing team and supporting staff who also will ask questions around GenAI in order to better understand the products and the field as a whole, but a lot more high level answers would likely be in order

**Below, please build your RAG pipeline including the relevant prompts. This is free form so you will need to create your own cells, text documentation as you need, etc.**

#### 3.3.0 Helper Functions

In [ ]:
# creating a custom output parser
class CustomOutputParser:
    def __call__(self, output):
        # Define a regular expression pattern to match the "Assistant:" line
        pattern = r"Assistant:\s*(.*)"

        # Search for the pattern in the output
        match = re.search(pattern, output)

        if match:
            # Return the captured group (the actual answer)
            return match.group(1).strip()

        # If no match is found, return None
        return None

custom_output_parser = CustomOutputParser()

# creating a document formatter
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### 3.3.1 LLM Configurations with Temperatures

##### Functions to generate LLMs given hyperparameters

In [ ]:
# function to build a cohere chat model
def build_cohere_llm_lc():
    COHERE_API_KEY = userdata.get('COHERE_API_KEY')
    cohere_chat_model = ChatCohere(cohere_api_key=COHERE_API_KEY)
    return cohere_chat_model

# function to build a mistral LLM in the LangChain environment
def build_mistral_llm_lc(max_new_tokens = 500,temperature = 0.6,top_p = 0.95,do_sample = True,repetition_penalty = 1.2):
  quantization_config = BitsAndBytesConfig(load_in_4bit=True, )
  llm_mistral_model = AutoModelForCausalLM.from_pretrained(
      "mistralai/Mistral-7B-Instruct-v0.2",
      torch_dtype=torch.float32,
      device_map='auto',
      quantization_config=quantization_config)
  llm_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
  mistral_pipe = pipeline(
      "text-generation",
      model=llm_mistral_model, # specifying the pre-trained model to use for text generation
      tokenizer=llm_mistral_tokenizer, # specifying the tokenizer
      max_new_tokens=max_new_tokens, # sets the maximum number of new tokens to generate.
      temperature=temperature, # controls the randomness of predictions
      top_p=top_p, # generates more coherent text
      do_sample=do_sample, # enables sampling
      repetition_penalty=repetition_penalty # penalizes repeated tokens to avoid redundancy in the generated text.
  )
  # setting the padding token ID
  mistral_pipe.model.config.pad_token_id = mistral_pipe.model.config.eos_token_id
  mistral_llm_lc = HuggingFacePipeline(pipeline=mistral_pipe)
  return mistral_llm_lc


##### Mistral with temperature = 0.2

In [ ]:
# naming a variable for the mistral llm with temperature 0.2
mistral_llm1_lc = build_mistral_llm_lc(temperature = 0.2)

##### Mistral with temperature 0.6

In [ ]:
# naming a variable for the mistral llm with temperature 0.6
mistral_llm2_lc = build_mistral_llm_lc(temperature = 0.6)

##### Cohere

In [ ]:
# naming a variable for the cohere chat model
cohere_llm_lc = build_cohere_llm_lc()

##### Function to get the LLM given the LLM model name and the temperature

In [ ]:
# function to get the LLM variable given the model name and temperature
def get_llm(llm_type, temperature):
  if llm_type == 'mistral':
    if temperature == 0.2:
      return mistral_llm1_lc
    if temperature == 0.6:
      return mistral_llm2_lc
  if llm_type == 'cohere':
      # for cohere temperature can be None
      return cohere_llm_lc


#### 3.3.2 Prompts

##### Prompt A_eng and Prompt A_mkt

In [ ]:
# writing the set of version A prompts
prompt_a_eng = """[INST]
You are a helpful question answering assistant specializing in NLP and GenAI. Your audience is a team of research engineers at a tech company that wants to roll out a new series of GenAI products.
These engineers understand NLP on a technical level, so they will need answers that provide appropriate technical depth and detail.
Please answer the engineers' questions using only the following context:

{context}


That is it for the context.

Now, you will answer the following question using the context provided in less than 100 words. Remember, you are answering the question for engineers who collectively have a good understanding of NLP already and require detailed and comprehensive answers. While your response should be comprehensive, please use the context provided above as your main source of information and keep it as concise as appropriate.

Answer the question directly in appropriate detail without any lists or bullet points.

Here is the question:

{question}
[/INST]

Assistant:"""

prompt_a_mkt = """[INST]
You are a helpful question answering assistant specializing in NLP and GenAI. Your audience is a marketing team at a tech company that wants to roll out a new series of GenAI products.
This marketing team does not understand NLP on a technical level, so they will not understand answers with overly technical terminology.
Please answer the marketing team's questions using only the following context:

{context}


That is it for the context.

Now, you will answer the following question using the context provided in less than 75 words. Remember, you are answering the question for a marketing team that collectively does not have a deep understanding of NLP. The purpose of your answer will be to provide the marketing team with high level information. Your answers will help the marketing team better understand their tech company's GenAI products and the GenAI field as a whole. While your response should be high level, please use the context provided above as your main source of information and keep it as concise as appropriate.

Answer the question directly in appropriate detail without any lists or bullet points.

Here is the question:

{question}
[/INST]

Assistant:"""



##### Prompt B_eng and Prompt B_mkt

In [ ]:
# writing the set of version B prompts
prompt_b_eng = """[INST]
You are a highly knowledgeable question-answering assistant specializing in the fields of Natural Language Processing (NLP) and Generative AI (GenAI). Your audience consists of a team of research engineers at a tech company preparing to launch a new series of GenAI products. These engineers have a strong technical background in NLP, so your responses must provide appropriate technical depth and detail.

Please answer the engineers' questions, only using the following context to inform your answer:

{context}

That is the context.

Now, answer the following question in less than 100 words, ensuring your response is detailed, comprehensive, and technically robust, tailored to engineers with a solid understanding of NLP.

Your response should use the context provided above as its main source of information. Answer the question directly, without any prefatory or introductory phrases. Provide a comprehensive, continuous answer, that utilizes the context provided above, without resorting to lists or bullet points. Again, keep your answer within the ~100 word limit and respond in paragraph form.

Here is the question:

{question}
[/INST]

Assistant:"""

prompt_b_mkt = """[INST]
You are a highly knowledgeable question-answering assistant specializing in Natural Language Processing (NLP) and Generative AI (GenAI). Your audience is a marketing team at a tech company preparing to launch a new series of GenAI products. This marketing team does not have a deep technical understanding of NLP, so avoid using overly technical terminology, and avoid going into highly technical depth and detail.

Please answer the marketing team's questions, only using the following context to inform your answer:

{context}

That is the context.

Now, answer the following question in less than 75 words, ensuring your response is high-level enough to be understood by a non-technical audience. Your goal is to help them better understand their company's GenAI products and the GenAI field in general. Your response should use the context provided above as its main source of information.

Your response should use the context provided above as its main source of information. Answer the question directly, without any prefatory or introductory phrases. Provide a comprehensive, continuous answer, that utilizes the context provided above, without resorting to lists or bullet points. Again, keep your answer within the ~75 word limit and respond in paragraph form.

Here is the question:

{question}
[/INST]

Assistant:"""



##### Prompt C_eng and Prompt C_mkt

In [ ]:
# writing the set of version C prompts
prompt_c_eng = """[INST]
You are a question answering assistant specializing in Natural Language Processing (NLP) and Generative AI (GenAI). Your job is to assist a team of research engineers at a tech company that is launching a new series of GenAI products.
These engineers understand NLP on a technical level, so they will need answers that provide appropriate technical depth and detail. You may want to use the following context when generating your response. Here is the context:
{context}
Now that you have some context, you will answer the following question in less than 100 words in a single paragraph format.
Do not use prefatory or introductory clauses. Do not use bullet points or numbered lists.
Just provide around 3 to 5 sentences, answering the question directly.
Remember, you are answering the question for engineers who collectively have a good understanding of NLP and require answers with technical detail. While your response should be comprehensive, please keep your answer appropriately concise and relevant to the question.
Here is the question:
{question}
[/INST]
Assistant:"""

prompt_c_mkt = """[INST]
You are a question answering assistant specializing in Natural Language Processing (NLP) and Generative AI (GenAI). Your job is to assist a marketing team at a tech company that is launching a new series of GenAI products.
This marketing team does not understand NLP on a technical level, so they will need answers that provide an appropriate level of high-level detail. You may want to use the following context when generating your response. Here is the context:
{context}
Now that you have some context, you will answer the following question in less than 75 words in a single paragraph format.
Do not use prefatory or introductory clauses. Do not use bullet points or numbered lists.
Just provide around 1 to 3 sentences, answering the question directly.
Remember, you are answering the question for a marketing team that collectively does not have a deeply technical understanding of NLP and requires high-level answers with only appropriate technical detail. While your response should be high-level and concise, please keep your answer appropriately detailed and relevant to the question.
Here is the question:
{question}
[/INST]
Assistant:"""

##### Function to get the RAG prompt given the prompt version letter and the 3 letter indicator for audience

In [ ]:
# function to get the RAG prompt given the prompt version letter and the 3 letter indicator for audience
def get_rag_prompt(prompt, audience):
  if prompt=='A' and audience=='eng':
    rag_prompt = prompt_a_eng
    return rag_prompt
  if prompt=='A' and audience=='mkt':
    rag_prompt = prompt_a_mkt
    return rag_prompt
  if prompt=='B' and audience=='eng':
    rag_prompt = prompt_b_eng
    return rag_prompt
  if prompt=='B' and audience=='mkt':
    rag_prompt = prompt_b_mkt
    return rag_prompt
  if prompt=='C' and audience=='eng':
    return prompt_c_eng
  if prompt=='C' and audience=='mkt':
    return prompt_c_mkt
  else:
    print('prompt must be one of A or B or C and audience must be one of eng or mkt.')


#### 3.3.3 Embedding Models

##### Setting the variable for the "multi-qa-mpnet-base-dot-v1" embedding model

In [ ]:
# setting the variable for the "multi-qa-mpnet-base-dot-v1" embedding model
base_embeddings1 = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

##### Setting the variable for the "all-distilroberta-v1" embedding model

In [ ]:
# setting the variable for the "all-distilroberta-v1" embedding model
base_embeddings2 = HuggingFaceEmbeddings(model_name="all-distilroberta-v1")

#### 3.3.3 Retrievers

##### Function to get Retriever given Chunk Parameters

In [ ]:
# function to get the retriever given the chunking parameters
def build_retriever(chunk_size, overlap, base_embeddings):
    loader = WebBaseLoader(
        web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header"))),)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    splits = text_splitter.split_documents(documents)
    vectorstore = Qdrant.from_documents(splits,
        base_embeddings,
        location=":memory:",
        collection_name="test2",)
    retriever = vectorstore.as_retriever()
    qdrant_vectorstore = Qdrant.from_documents(splits,
        base_embeddings,
        location=":memory:",
        collection_name="rag_tech_db2",
        force_recreate=True)
    retriever = qdrant_vectorstore.as_retriever()

    global_doc_number = 1

    arxiv_numbers = ('2005.11401', '2104.07567', '2104.09864', '2105.03011', '2106.09685', '2203.02155', '2211.09260', '2211.12561',
                    '2212.09741', '2305.14314', '2305.18290', '2306.15595', '2309.08872', '2309.15217', '2310.06825', '2310.11511',
                    '2311.08377', '2312.05708', '2401.06532', '2401.17268', '2402.01306', '2402.19473', '2406.04744')
    all_arxiv_pages = []
    for identifier in arxiv_numbers:
      arx_url = f"https://arxiv.org/pdf/{identifier}.pdf"
      arx_loader = PyMuPDFLoader(arx_url)
      arx_pages = arx_loader.load()
      for page_num in range(len(arx_pages)):
          page = arx_pages[page_num]
          page.metadata['page_num'] = page_num
          page.metadata['doc_num'] = global_doc_number
          page.metadata['doc_source'] = "ArXiv"
          all_arxiv_pages.append(page)
    splits = text_splitter.split_documents(all_arxiv_pages)
    for idx, text in enumerate(splits):
        splits[idx].metadata['split_id'] = idx
    qdrant_vectorstore.add_documents(documents=splits)

    global_doc_number += 1

    wiki_docs = WikipediaLoader(query="Generative Artificial Intelligence", load_max_docs=4).load()
    for idx, text in enumerate(wiki_docs):
        wiki_docs[idx].metadata['doc_num'] = global_doc_number
        wiki_docs[idx].metadata['doc_source'] = "Wikipedia"
    wiki_splits = text_splitter.split_documents(wiki_docs)
    for idx, text in enumerate(wiki_splits):
        wiki_splits[idx].metadata['split_id'] = idx
    qdrant_vectorstore.add_documents(documents=wiki_splits)

    global_doc_number += 1

    wiki_docs = WikipediaLoader(query="Information Retrieval", load_max_docs=4).load()
    for idx, text in enumerate(wiki_docs):
        wiki_docs[idx].metadata['doc_num'] = global_doc_number
        wiki_docs[idx].metadata['doc_source'] = "Wikipedia"
    wiki_splits = text_splitter.split_documents(wiki_docs)
    for idx, text in enumerate(wiki_splits):
        wiki_splits[idx].metadata['split_id'] = idx
    qdrant_vectorstore.add_documents(documents=wiki_splits)

    global_doc_number += 1

    wiki_docs = WikipediaLoader(query="Large Language Models", load_max_docs=4).load()
    for idx, text in enumerate(wiki_docs):
        wiki_docs[idx].metadata['doc_num'] = global_doc_number
        wiki_docs[idx].metadata['doc_source'] = "Wikipedia"
    wiki_splits = text_splitter.split_documents(wiki_docs)
    for idx, text in enumerate(wiki_splits):
        wiki_splits[idx].metadata['split_id'] = idx
    qdrant_vectorstore.add_documents(documents=wiki_splits)

    global_doc_number += 1

    web_loader = WebBaseLoader(
        web_paths=("https://lilianweng.github.io/posts/2020-10-29-odqa/",
                  "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
                  "https://lilianweng.github.io/posts/2018-06-24-attention/",
                  "https://lilianweng.github.io/posts/2023-06-23-agent/",
                  "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"),
        bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
    web_documents = web_loader.load()
    for idx, text in enumerate(web_documents):
        web_documents[idx].metadata['doc_num'] = global_doc_number
        web_documents[idx].metadata['doc_source'] = "WWW"
    web_splits = text_splitter.split_documents(web_documents)
    for idx, text in enumerate(web_splits):
        web_splits[idx].metadata['split_id'] = idx
    qdrant_vectorstore.add_documents(documents=web_splits)

    retriever = qdrant_vectorstore.as_retriever()
    return retriever

##### Chunk Size 128 with 0 overlap, with both embedding models

In [ ]:
# setting variables for the 128/0 chunking parameters for the first embedding model
retriever1_1 = build_retriever(chunk_size=128, overlap=0, base_embeddings=base_embeddings1)

In [ ]:
# setting variables for the 128/0 chunking parameters for the second embedding model
retriever1_2 = build_retriever(chunk_size=128, overlap=0, base_embeddings=base_embeddings2)

##### Chunk Size 300 with 40 overlap, with both embedding models

In [ ]:
# setting variables for the 300/40 chunking parameters for the first embedding model
retriever2_1 = build_retriever(chunk_size=300, overlap=40, base_embeddings=base_embeddings1)

In [ ]:
# setting variables for the 300/40 chunking parameters for the second embedding model
retriever2_2 = build_retriever(chunk_size=300, overlap=40, base_embeddings=base_embeddings2)

##### Function to get the Retriever given the embedding model, chunk size, and chunk overlap

In [ ]:
# function to get the retriever given the embedding model, chunk size, and chunk overlap
def get_retriever(embedding_model, chunk_size, overlap):
  if embedding_model=='multi-qa-mpnet-base-dot-v1' and chunk_size==128 and overlap==0:
    retriever = retriever1_1
    return retriever
  if embedding_model=='multi-qa-mpnet-base-dot-v1' and chunk_size==300 and overlap==40:
    retriever = retriever2_1
    return retriever
  if embedding_model=='all-distilroberta-v1' and chunk_size==128 and overlap==0:
    retriever = retriever1_2
    return retriever
  if embedding_model=='all-distilroberta-v1' and chunk_size==300 and overlap==40:
    retriever = retriever2_2
    return retriever
  else:
    print('embedding_model must be one of multi-qa-mpnet-base-dot-v1 or all-distilroberta-v1')


#### 3.3.4 RAG Chains

##### Functions to build the RAG chains

In [ ]:
# function to build the mistral rag chain given the retriever, prompt, and LLM
def build_rag_chain_mistral(retriever, rag_prompt, llm_lc):
    rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)
    rag_chain = (
      {"context": retriever | format_docs,
      "question": RunnablePassthrough()}
      | rag_prompt_template
      | llm_lc
      | custom_output_parser
    )
    return rag_chain

# function to build the cohere rag chain given the retriever, prompt, and LLM
def build_rag_chain_cohere(retriever, rag_prompt, llm_lc):
    output_parser = StrOutputParser()
    rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)
    test_cohere_llm_chain_short = (
       {"context": retriever | format_docs,
       "question": RunnablePassthrough()}
        | rag_prompt_template
        | llm_lc
        | output_parser
       )
    return test_cohere_llm_chain_short

# function to build the rag chain given the model name, embedding model name, prompt version letter, audience 3-letter indicator, temperature, chunk_size, and overlap
def build_rag_chain(model='mistral', embedding_model='multi-qa-mpnet-base-dot-v1', prompt='A', audience='eng', temperature=0.2, chunk_size=128, overlap=0):
  retriever = get_retriever(embedding_model, chunk_size, overlap)
  rag_prompt = get_rag_prompt(prompt, audience)
  llm_lc = get_llm(model, temperature)

  if model=='cohere':
    chain=build_rag_chain_cohere(retriever, rag_prompt, llm_lc)
  if model=='mistral':
    llm_lc= build_mistral_llm_lc(temperature=temperature)
    chain=build_rag_chain_mistral(retriever, rag_prompt, llm_lc)
  return chain

##4. Tests & Evaluations

Here you should evaluate the results. First, you should define your evaluation metrics and then you should run evaluation tests. This is really your area, but key results to show are:

1) Your metrics of choice  
2) How  your various models compare to the labeled validation data.

Make sure you look at the results for the marketing team and the research team separately.

**Note:** You do not need to run all models against all labeled questions, as that may take some time. Just do that for a few models/configs, and test a larger set with a smaller subset.

**This is free form so you will need to create your own cells, text documentation as you need, etc.**



### 4.1. Metrics

Please define and motivate your metrics here. Please feel free to add more text and code cells as needed.

1. The first metric I will be using is simple cosine similarty
* This metric will measure the overall similarity between the RAG answer and the gold standard answer.

In [ ]:
def get_cosine_similarity(gold_standard, llm_answer):
  g_s_embedded = base_embeddings2.embed_query(gold_standard)
  llm_a_embedded = base_embeddings2.embed_documents([llm_answer])
  cos_sim = cosine_similarity([g_s_embedded], llm_a_embedded)[0][0]
  return cos_sim

2. The second metric I will be using is the BERTScore metric
* The BERTScore metric first creates embedding representations of the RAG answer and the gold standard answer
* Then, it generates similarity scores between the embeddings using cosine similarity
* Then, each token in the RAG generated sentence is matched with its most similar token in the gold standard answer
* Then, rare word importance is considered using Inverse Document Frequency
* Finally, the score is rescaled to an interpretable range

In [ ]:
# BERTScore precision, recall, F1 score
def get_bert_score(gold_standard, llm_answer):
  scorer = BERTScorer(model_type='bert-base-uncased')
  P, R, F1 = scorer.score([llm_answer], [gold_standard])
  return {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

In [ ]:
def create_eval_df(val_df, audience):
  if audience == 'eng':
    gold_col = 'gold_answer_research'
    rag_col = 'rag_answer_research'
  if audience == 'mkt':
    gold_col = 'gold_answer_marketing'
    rag_col = 'rag_answer_marketing'
  val_df['cos_similarity'] = val_df.apply(lambda row: get_cosine_similarity(row[gold_col],row[rag_col]),axis=1)
  val_df_bert = pd.json_normalize(val_df.apply(lambda row: get_bert_score(row[gold_col],row[rag_col]),axis=1))
  val_df['bert_precision'] = val_df_bert['precision']
  val_df['bert_recall'] = val_df_bert['recall']
  val_df['bert_f1'] = val_df_bert['f1']
  return val_df


In [ ]:
def create_overall_metrics_df(val_df_eval, pipeline_num, audience):
  return pd.DataFrame({'pipeline': [pipeline_num],
                       'audience': [audience],
                       'avg_cos_similarity':[val_df_eval.cos_similarity.mean()],
                       'avg_bert_precision':[val_df_eval.bert_precision.mean()],
                       'avg_bert_recall':[val_df_eval.bert_recall.mean()],
                       'avg_bert_f1':[val_df_eval.bert_f1.mean()],
                       'combined_score':[(val_df_eval.cos_similarity.mean() + val_df_eval.bert_f1.mean())/2]
                       })

### 4.2. Evaluation Comparisons

Document your key runs here. Feel free to add more text and code cells as needed.

In [ ]:
val_df = pd.DataFrame(validation_questions_answers).T

#### 4.2.2 RAG pipelines


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


##### Pipeline 1
* model='Cohere'
* embedding_model='multi-qa-mpnet-base-dot-v1'
* Prompt A
* chunk_size=128, overlap=0


In [ ]:
# build rag given configuration
rag_pipe_1_eng = build_rag_chain(model='cohere',
                embedding_model='multi-qa-mpnet-base-dot-v1',
                prompt='A',
                audience='eng',
                temperature=None,
                chunk_size=128,
                overlap=0)

rag_pipe_1_mkt = build_rag_chain(model='cohere',
                embedding_model='multi-qa-mpnet-base-dot-v1',
                prompt='A',
                audience='mkt',
                temperature=None,
                chunk_size=128,
                overlap=0)

###### Engineering

In [ ]:
val_df1_eng = val_df.copy()

In [ ]:
rag1_answers_research = []
for question in val_df1_eng.iloc[:10,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[10:20,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[20:30,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[30:40,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[40:50,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[50:60,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[60:70,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
for question in val_df1_eng.iloc[70:75,:].question:
  answer = rag_pipe_1_eng.invoke(question)
  rag1_answers_research.append(answer)

In [ ]:
val_df1_eng['rag_answer_research'] = rag1_answers_research

In [ ]:
val_df1_eng.to_csv('/content/drive/My Drive/290/rag_answers_eng/rag1_answers_eng.csv')

###### Evaluating Engineering

In [ ]:
val_df1_eng = pd.read_csv('/content/drive/My Drive/290/rag_answers_eng/rag1_answers_eng.csv',index_col=0)

In [ ]:
val_df1_eng_eval = create_eval_df(val_df1_eng, 'eng')

In [ ]:
val_df1_eng_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_research,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,Large language models (LLMs) have become a cor...,0.727397,0.547440,0.675203,0.604646
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,"Large language models, also known as transform...",0.681756,0.586434,0.641642,0.612797
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,Large language models have evolved through var...,0.721333,0.630558,0.667987,0.648733
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.",Sure! Here are some examples of large language...,0.647905,0.533290,0.670013,0.593884
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,The distribution and usage rights of source-av...,0.715254,0.652477,0.725590,0.687094


In [ ]:
val_df1_eng_overall_metrics = create_overall_metrics_df(val_df1_eng_eval, 1, 'eng')
val_df1_eng_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,1,eng,0.743264,0.626787,0.663099,0.642672,0.692968


###### Marketing

In [ ]:
val_df1_mkt = val_df.copy()

In [ ]:
rag1_answers_marketing = []
for question in val_df1_mkt.iloc[:10,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[10:20,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[20:30,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[30:40,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[40:50,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[50:60,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[60:70,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
for question in val_df1_mkt.iloc[70:75,:].question:
  answer = rag_pipe_1_mkt.invoke(question)
  rag1_answers_marketing.append(answer)

In [ ]:
val_df1_mkt['rag_answer_marketing'] = rag1_answers_marketing

In [ ]:
val_df1_mkt.to_csv('/content/drive/My Drive/290/rag_answers_mkt/rag1_answers_mkt.csv')

###### Evaluating Marketing

In [ ]:
val_df1_mkt = pd.read_csv('/content/drive/My Drive/290/rag_answers_mkt/rag1_answers_mkt.csv',index_col=0)

In [ ]:
val_df1_mkt_eval = create_eval_df(val_df1_mkt, 'mkt')

In [ ]:
val_df1_mkt_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_marketing,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,Large language models are an essential compone...,0.808528,0.514836,0.682134,0.586793
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,"Large language models, the foundation of GenAI...",0.628176,0.503207,0.604558,0.549246
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,"Large language models, which are at the core o...",0.649927,0.534834,0.579263,0.556163
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.",Some well-known large language models and thei...,0.468835,0.342970,0.563165,0.426313
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,The distribution and availability of language ...,0.423500,0.524156,0.629998,0.572224


In [ ]:
val_df1_mkt_overall_metrics = create_overall_metrics_df(val_df1_mkt_eval, 1, 'mkt')
val_df1_mkt_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,1,mkt,0.707769,0.558472,0.672561,0.608451,0.65811


##### Pipeline 2
* model='Mistral'
* embedding_model='all-distilroberta-v1'
* Prompt A
* temperature=0.2, max_new_tokens=500
* chunk_size=128, overlap=0

In [ ]:
# build rag given configuration
rag_pipe_2_eng = build_rag_chain(model='mistral',
                embedding_model='all-distilroberta-v1',
                prompt='A',
                audience='eng',
                temperature=0.2,
                chunk_size=128,
                overlap=0)

rag_pipe_2_mkt = build_rag_chain(model='mistral',
                embedding_model='all-distilroberta-v1',
                prompt='A',
                audience='mkt',
                temperature=0.2,
                chunk_size=128,
                overlap=0)

###### Engineering

In [ ]:
val_df2_eng = val_df.copy()

In [ ]:
rag2_answers_research = []
for question in val_df2_eng.iloc[:10,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_eng.iloc[10:20,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_eng.iloc[20:30,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_eng.iloc[30:40,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)

In [ ]:
for question in val_df2_eng.iloc[40:50,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_eng.iloc[50:60,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_eng.iloc[60:70,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_eng.iloc[70:75,:].question:
  answer = rag_pipe_2_eng.invoke(question)
  rag2_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
val_df2_eng['rag_answer_research'] = rag2_answers_research

In [ ]:
val_df2_eng.to_csv('/content/drive/My Drive/290/rag_answers_eng/rag2_answers_eng.csv')

###### Evaluating Engineering

In [ ]:
val_df2_eng = pd.read_csv('/content/drive/My Drive/290/rag_answers_eng/rag2_answers_eng.csv',index_col=0)

In [ ]:
val_df2_eng_eval = create_eval_df(val_df2_eng, 'eng')

In [ ]:
val_df2_eng_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_research,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,"In Natural Language Processing (NLP), Large La...",0.778479,0.674721,0.666606,0.670639
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,"In the context provided, large language models...",0.756729,0.621127,0.607627,0.614303
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,The context suggests that large language model...,0.656299,0.574878,0.565427,0.570113
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.","In the late 1990s, IBM introduced Deep Blue, n...",0.658438,0.559185,0.681116,0.614157
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,"In the context provided, no specific details a...",0.900623,0.680862,0.658643,0.669568


In [ ]:
val_df2_eng_overall_metrics = create_overall_metrics_df(val_df2_eng_eval, 2, 'eng')
val_df2_eng_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,2,eng,0.750531,0.631623,0.633604,0.631663,0.691097


###### Marketing

In [ ]:
val_df2_mkt = val_df.copy()

In [ ]:
rag2_answers_marketing = []
for question in val_df2_mkt.iloc[:10,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[10:20,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[20:30,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[30:40,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[40:50,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[50:60,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[60:70,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df2_mkt.iloc[70:75,:].question:
  answer = rag_pipe_2_mkt.invoke(question)
  rag2_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
val_df2_mkt['rag_answer_marketing'] = rag2_answers_marketing

In [ ]:
val_df2_mkt.to_csv('/content/drive/My Drive/290/rag_answers_mkt/rag2_answers_mkt.csv')

###### Evaluating Marketing

In [ ]:
val_df2_mkt = pd.read_csv('/content/drive/My Drive/290/rag_answers_mkt/rag2_answers_mkt.csv',index_col=0)

In [ ]:
val_df2_mkt_eval = create_eval_df(val_df2_mkt, 'mkt')

In [ ]:
val_df2_mkt_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_marketing,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,Large language models are powerful tools in Na...,0.841659,0.578948,0.683473,0.626883
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,A large language model learns by analyzing vas...,0.770946,0.623596,0.654722,0.638780
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,"Based on the context provided, large language ...",0.668196,0.537000,0.528852,0.532895
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.","Absolutely. Before 2017, notable large languag...",0.314909,0.335027,0.365910,0.349788
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,The distribution of source-available language ...,0.314550,0.642178,0.674476,0.657931


In [ ]:
val_df2_mkt_overall_metrics = create_overall_metrics_df(val_df2_mkt_eval, 2, 'mkt')
val_df2_mkt_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,2,mkt,0.727712,0.597814,0.653781,0.623113,0.675412


##### Pipeline 3
* model='Mistral'
* embedding_model='multi-qa-mpnet-base-dot-v1'
* Prompt B
* temperature=0.6
* chunk_size=300, overlap=40

In [ ]:
# build rag given configuration
rag_pipe_3_eng = build_rag_chain(model='mistral',
                embedding_model='multi-qa-mpnet-base-dot-v1',
                prompt='B',
                audience='eng',
                temperature=0.6,
                chunk_size=300,
                overlap=40)

rag_pipe_3_mkt = build_rag_chain(model='mistral',
                embedding_model='multi-qa-mpnet-base-dot-v1',
                prompt='B',
                audience='mkt',
                temperature=0.6,
                chunk_size=300,
                overlap=40)

###### Engineering

In [ ]:
val_df3_eng = val_df.copy()

In [ ]:
rag3_answers_research = []
for question in val_df3_eng.iloc[:10,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_eng.iloc[10:20,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_eng.iloc[20:30,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_eng.iloc[30:40,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)

In [ ]:
for question in val_df3_eng.iloc[40:50,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_eng.iloc[50:60,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_eng.iloc[60:70,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_eng.iloc[70:75,:].question:
  answer = rag_pipe_3_eng.invoke(question)
  rag3_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
val_df3_eng['rag_answer_research'] = rag3_answers_research

In [ ]:
val_df3_eng.to_csv('/content/drive/My Drive/290/rag_answers_eng/rag3_answers_eng.csv')

###### Evaluating Engineering

In [ ]:
val_df3_eng = pd.read_csv('/content/drive/My Drive/290/rag_answers_eng/rag3_answers_eng.csv',index_col=0)

In [ ]:
val_df3_eng_eval = create_eval_df(val_df3_eng, 'eng')

In [ ]:
val_df3_eng_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_research,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,In the realm of Natural Language Processing (N...,0.883840,0.612936,0.675153,0.642542
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,"In the context provided, we don't find an exac...",0.770484,0.568004,0.564551,0.566273
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,The context suggests that large language model...,0.762287,0.580499,0.590225,0.585322
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.","In the context provided, we find mentions of s...",0.690561,0.450663,0.543817,0.492877
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,"Given the context, there seems to be no clear ...",0.745583,0.685182,0.633318,0.658230


In [ ]:
val_df3_eng_overall_metrics = create_overall_metrics_df(val_df3_eng_eval, 3, 'eng')
val_df3_eng_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,3,eng,0.781674,0.610423,0.644359,0.625765,0.70372


###### Marketing

In [ ]:
val_df3_mkt = val_df.copy()

In [ ]:
rag3_answers_marketing = []
for question in val_df3_mkt.iloc[:10,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_mkt.iloc[10:20,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_mkt.iloc[20:30,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_mkt.iloc[30:40,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_mkt.iloc[40:50,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
rag3_answers_marketing=val_df3_mkt.rag_answer_marketing.to_list()

In [ ]:
for question in val_df3_mkt.iloc[50:60,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_mkt.iloc[60:70,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df3_mkt.iloc[70:75,:].question:
  answer = rag_pipe_3_mkt.invoke(question)
  rag3_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
val_df3_mkt['rag_answer_marketing'] = rag3_answers_marketing

In [ ]:
val_df3_mkt.to_csv('/content/drive/My Drive/290/rag_answers_mkt/rag3_answers_mkt.csv')

###### Evaluating Marketing

In [ ]:
val_df3_mkt = pd.read_csv('/content/drive/My Drive/290/rag_answers_mkt/rag3_answers_mkt.csv',index_col=0)

In [ ]:
val_df3_mkt_eval = create_eval_df(val_df3_mkt, 'mkt')

In [ ]:
val_df3_mkt_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_marketing,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,Large language models are powerful tools in Na...,0.859934,0.530634,0.582350,0.555291
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,"A large language model, such as those used in ...",0.737151,0.545694,0.653399,0.594709
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,"Large language models, such as those mentioned...",0.690080,0.562414,0.593334,0.577460
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.","In recent years, large-scale language models l...",0.493731,0.348702,0.456494,0.395383
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,In the realm of General Artificial Intelligenc...,0.439620,0.589573,0.604727,0.597054


In [ ]:
val_df3_mkt_overall_metrics = create_overall_metrics_df(val_df3_mkt_eval, 3, 'mkt')
val_df3_mkt_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,3,mkt,0.756638,0.554833,0.660372,0.601319,0.678979


##### Pipeline 4
* model='Cohere'
* embedding_model='all-distilroberta-v1'
* Prompt C
* chunk_size=300, overlap=40

In [ ]:
# build rag given configuration
rag_pipe_4_eng = build_rag_chain(model='cohere',
                embedding_model='all-distilroberta-v1',
                prompt='C',
                audience='eng',
                temperature=None,
                chunk_size=300,
                overlap=40)

rag_pipe_4_mkt = build_rag_chain(model='cohere',
                embedding_model='all-distilroberta-v1',
                prompt='C',
                audience='mkt',
                temperature=None,
                chunk_size=300,
                overlap=40)

###### Engineering

In [ ]:
val_df4_eng = val_df.copy()

In [ ]:
rag4_answers_research = []
for question in val_df4_eng.iloc[:10,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[10:20,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[20:30,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[30:40,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[40:50,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[50:60,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[60:70,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_eng.iloc[70:75,:].question:
  answer = rag_pipe_4_eng.invoke(question)
  rag4_answers_research.append(answer)
  torch.cuda.empty_cache()

In [ ]:
val_df4_eng['rag_answer_research'] = rag4_answers_research

In [ ]:
val_df4_eng.to_csv('/content/drive/My Drive/290/rag_answers_eng/rag4_answers_eng.csv')

###### Evaluating Engineering

In [ ]:
val_df4_eng = pd.read_csv('/content/drive/My Drive/290/rag_answers_eng/rag4_answers_eng.csv',index_col=0)

In [ ]:
val_df4_eng_eval = create_eval_df(val_df4_eng, 'eng')

In [ ]:
val_df4_eng_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_research,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,Large language models (LLMs) have become pivot...,0.857932,0.675201,0.670343,0.672763
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,Large language models learn by analyzing vast ...,0.809942,0.654429,0.619647,0.636563
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,"Transformer-based architectures, such as the A...",0.652485,0.660419,0.635273,0.647602
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.",Some prominent examples of large language mode...,0.702908,0.588870,0.670439,0.627013
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,The choice of licensing models for source-avai...,0.744668,0.705258,0.715837,0.710508


In [ ]:
val_df4_eng_overall_metrics = create_overall_metrics_df(val_df4_eng_eval, 4, 'eng')
val_df4_eng_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,4,eng,0.773411,0.659659,0.662,0.6601,0.716755


###### Marketing

In [ ]:
val_df4_mkt = val_df.copy()

In [ ]:
rag4_answers_marketing = []
for question in val_df4_mkt.iloc[:10,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[10:20,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[20:30,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[30:40,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[40:50,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[50:60,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[60:70,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
for question in val_df4_mkt.iloc[70:75,:].question:
  answer = rag_pipe_4_mkt.invoke(question)
  rag4_answers_marketing.append(answer)
  torch.cuda.empty_cache()

In [ ]:
val_df4_mkt['rag_answer_marketing'] = rag4_answers_marketing

In [ ]:
val_df4_mkt.to_csv('/content/drive/My Drive/290/rag_answers_mkt/rag4_answers_mkt.csv')

###### Evaluating Marketing

In [ ]:
val_df4_mkt = pd.read_csv('/content/drive/My Drive/290/rag_answers_mkt/rag4_answers_mkt.csv',index_col=0)

In [ ]:
val_df4_mkt_eval = create_eval_df(val_df4_mkt, 'mkt')

In [ ]:
val_df4_mkt_eval.head()

,question,gold_answer_research,gold_answer_marketing,rag_answer_marketing,cos_similarity,bert_precision,bert_recall,bert_f1
0,What purpose do large language models serve in...,Large language models (LLMs) serve the purpose...,Large language models serve the purpose of imp...,Large language models (LLMs) are a pivotal too...,0.826107,0.563684,0.642879,0.600682
1,How does a large language model learn from tex...,A large language model learns from text during...,A large language model learns from text during...,Large language models learn by analyzing vast ...,0.683940,0.617790,0.660558,0.638459
2,What are some key architectures behind the dev...,Key architectures behind the development of la...,Key architectures behind the development of la...,Large language models (LLMs) are built on tran...,0.776285,0.609411,0.605061,0.607228
3,Can you name some specific large language mode...,Some specific large language models include GP...,"Chinchilla by DeepMind, GPT-3 by OpenAI.",Some well-known large language models include ...,0.504350,0.430028,0.632143,0.511856
7,What licensing models have been adopted for th...,"Based on the provided context, it seems that l...",Answer: Some organizations choose open-sourcin...,"Currently, there is no standard licensing mode...",0.246083,0.694451,0.720447,0.707211


In [ ]:
val_df4_mkt_overall_metrics = create_overall_metrics_df(val_df4_mkt_eval, 4, 'mkt')
val_df4_mkt_overall_metrics

,pipeline,audience,avg_cos_similarity,avg_bert_precision,avg_bert_recall,avg_bert_f1,combined_score
0,4,mkt,0.770439,0.631741,0.679037,0.652514,0.711477


## 5. Results




### 5.1 Model Specifications

Document the detailed specs of your choices. Also comment on how you valued the needs of the marketing team vs the needs of the researchers, in case you had to make a trade-off.

For my final model, I chose the Cohere model, the embedding model multi-qa-mpnet-base-dot-v1, chunk size 128 and overlap 0, and prompt A. In the experiments with various model configurations, I compared RAG outputs with the gold standard answers for each audience. The aforementioned model performed the best, given its combined average cosine similarity and BertScore F1 score, for both audiences, across experiments. Therefore, the needs of the marketing team and researchers are equally being valued in this choice.


### 5.2 Some Test Questions

**QUESTIONS:**


Please study the answers generated by your chosen setup for these specific test questions:

1. "What purpose do large language models serve in the field of natural language processing?" (Question 0)

2. "What methods are typically employed to create training data for embedding models that use task-specific instructions?" (Question 50)

3. "How does a model's ability to answer questions relate to its exposure to specific types of questions during training?" (Question 83, no labeled answers)

For each of the three questions above please provide:

a) The RAG results (research and marketing response)  
b) The context provided  
c) The document sources for the context  
d) Also discuss your metric(s) for the first two examples (for both responses) compared to the gold responses

Then, for questions 1 and 2, comment on how well you feel your metrics captured the differences and similarities between your answer and the gold answer?

Put your answers to these questions into the answers file as you have done on previous assignments. Please consult the answer file for further details.








In [ ]:
# helper functions
def extract_context(text):
    pattern = re.compile(r'Here is the context:(.*?)That is it for the context.', re.DOTALL)

    match = pattern.search(text)

    if match:
        return match.group(1).strip()
    else:
        return None

def final_rag_context_provided(question):
  final_retriever = get_retriever('all-distilroberta-v1', 300, 40)
  final_prompt = ChatPromptTemplate.from_template('[/INST]Here is the context:{context} That is it for the context.[INST]')
  final_rag_chain = (
    {"context": final_retriever | format_docs,
     "question": RunnablePassthrough()}
    | final_prompt
    )
  output = final_rag_chain.invoke(question)
  context = extract_context(output.messages[0].content)
  return context

def get_document_sources(question):
  vectorstore = get_retriever('all-distilroberta-v1', 300, 40).vectorstore
  docs = vectorstore.similarity_search_by_vector(base_embeddings2.embed_query(question)) # will rank the splits
  for doc in docs:
    print('document: ', doc.page_content)
    print('source: ', doc.metadata['source'])
    print('doc_source:', doc.metadata['doc_source'])
    print('\n')

In [ ]:
# function to build the final rag chain given the model name, embedding model name, prompt version letter, audience 3-letter indicator, temperature, chunk_size, and overlap
def build_rag_chain(model='mistral', embedding_model='multi-qa-mpnet-base-dot-v1', prompt='A', audience='eng', temperature=0.2, chunk_size=128, overlap=0):
  retriever = get_retriever(embedding_model, chunk_size, overlap)
  rag_prompt = get_rag_prompt(prompt, audience)
  llm_lc = get_llm(model, temperature)

  if model=='cohere':
    chain=build_rag_chain_cohere(retriever, rag_prompt, llm_lc)
  if model=='mistral':
    llm_lc= build_mistral_llm_lc(temperature=temperature)
    chain=build_rag_chain_mistral(retriever, rag_prompt, llm_lc)
  return chain

In [ ]:
# this is the final set of rag chains I chose
final_rag_pipe_eng = build_rag_chain(model='cohere',
                embedding_model='all-distilroberta-v1',
                prompt='C',
                audience='eng',
                temperature=None,
                chunk_size=300,
                overlap=40)

final_rag_pipe_mkt = build_rag_chain(model='cohere',
                embedding_model='all-distilroberta-v1',
                prompt='C',
                audience='mkt',
                temperature=None,
                chunk_size=300,
                overlap=40)

#### 5.2.1 Test Question 1

Please run the query:

In [ ]:
test_q1=validation_questions_answers[0]['question']

In [ ]:
gold_ans1_eng=validation_questions_answers[0]['gold_answer_research']

In [ ]:
gold_ans1_eng

'Large language models (LLMs) serve the purpose of enabling general-purpose language generation and other natural language processing tasks such as classification. They achieve this by learning statistical relationships from text documents during computationally intensive self-supervised and semi-supervised training. LLMs can be used for text generation by predicting the next token or word, making them valuable for tasks like speech recognition, machine translation, and information retrieval. Additionally, LLMs have superseded previous models like recurrent neural networks, showcasing their efficiency and effectiveness in NLP tasks.'

In [ ]:
gold_ans1_mkt=validation_questions_answers[0]['gold_answer_marketing']

In [ ]:
gold_ans1_mkt

'Large language models serve the purpose of improving performance in various natural language processing tasks, such as speech recognition, machine translation, natural language generation, optical character recognition, handwriting recognition, grammar induction, and information retrieval.'

a) The RAG results (research and marketing response)

In [ ]:
test_ans1_eng = final_rag_pipe_eng.invoke(test_q1)

In [ ]:
test_ans1_eng

'Large language models (LLMs) have become integral to natural language processing (NLP) due to their ability to learn and model complex statistical relationships from vast text corpora. This enables them to generate human-like language and perform advanced language-based tasks. The combination of large datasets, feedforward neural networks, and transformers has resulted in the current state-of-the-art performance of LLMs, surpassing that of recurrent neural network-based models and symbolic language models. They are now essential for developing advanced NLP applications and driving the field forward.'

In [ ]:
test_ans1_mkt = final_rag_pipe_mkt.invoke(test_q1)

In [ ]:
test_ans1_mkt

'Large language models (LLMs) are an integral part of natural language processing (NLP), as they enable machines to understand and generate human-like language. LLMs are trained on vast datasets, allowing them to learn the statistical relationships between words and generate meaningful responses, making them essential for building intelligent language-based applications.'

b) The context provided


In [ ]:
final_rag_context_provided(test_q1)

'A large language model (LLM) is a computational model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. Based on language models, LLMs acquire these abilities by learning statistical relationships from vast\n\nlanguage models dominated over symbolic language models, as they can usefully ingest large datasets.\n\nLarge language models, currently their most advanced form, are a combination of larger datasets (frequently using words scraped from the public internet), feedforward neural networks, and transformers. They have superseded recurrent neural network-based models, which had previously superseded the\n\n2School of Computer Science, Beijing University of Posts and Telecommunications\n3School of Artificial Intelligence, Nankai University, 4Beijing Academy of Artificial Intelligence\nyutaozhu94@gmail.com, dou@ruc.edu.cn\nAbstract\nLarge language models (LLMs) have demon-'

c) The document sources for the context


In [ ]:
get_document_sources(test_q1)

document:  A large language model (LLM) is a computational model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. Based on language models, LLMs acquire these abilities by learning statistical relationships from vast
source:  https://en.wikipedia.org/wiki/Large_language_model
doc_source: Wikipedia


document:  language models dominated over symbolic language models, as they can usefully ingest large datasets.
source:  https://en.wikipedia.org/wiki/Large_language_model
doc_source: Wikipedia


document:  Large language models, currently their most advanced form, are a combination of larger datasets (frequently using words scraped from the public internet), feedforward neural networks, and transformers. They have superseded recurrent neural network-based models, which had previously superseded the
source:  https://en.wikipedia.org/wiki/Language_model
doc_source: Wikipedia


document:  2School of Comp

d) Also discuss your metric(s) for the first two examples (for both responses) compared to the gold responses

In [ ]:
eng_cos_sim1=get_cosine_similarity(gold_ans1_eng,test_ans1_eng)
eng_bert_score1=get_bert_score(gold_ans1_eng,test_ans1_eng)
eng_combined_score1=(eng_cos_sim1+eng_bert_score1['f1'])/2

mkt_cos_sim1=get_cosine_similarity(gold_ans1_mkt,test_ans1_mkt)
mkt_bert_score1=get_bert_score(gold_ans1_mkt,test_ans1_mkt)
mkt_combined_score1=(mkt_cos_sim1+mkt_bert_score1['f1'])/2

In [ ]:
print('Engineering Evaluation:')
print('cosine similarity:', eng_cos_sim1)
print('bert score:', eng_bert_score1)
print('combined score:', eng_combined_score1)
print('\n\n')

print('Marketing Evaluation:')
print('cosine similarity:', mkt_cos_sim1)
print('bert score:', mkt_bert_score1)
print('combined score:', mkt_combined_score1)

Engineering Evaluation:
cosine similarity: 0.9100002597590183
bert score: {'precision': 0.6556969881057739, 'recall': 0.6663129329681396, 'f1': 0.6609623432159424}
combined score: 0.7854813014874804



Marketing Evaluation:
cosine similarity: 0.738350294083111
bert score: {'precision': 0.5388528108596802, 'recall': 0.586467981338501, 'f1': 0.5616530179977417}
combined score: 0.6500016560404264


Discuss...

#### 5.2.2 Test Question 2

Please run the query:

In [ ]:
test_q2=validation_questions_answers[50]['question']

In [ ]:
gold_ans2_eng=validation_questions_answers[50]['gold_answer_research']

In [ ]:
gold_ans2_eng

'To create training data for embedding models that use task-specific instructions, a common method is to combine datasets from different sources, such as the SuperNaturalInstructions dataset with existing collections designed for embedding training. The SuperNaturalInstructions dataset provides natural language instructions, which can be paired with positive and negative examples to form training samples. Additionally, for tasks like classification or similarity, training samples can be constructed by selecting text sequences associated with different classes or similarities. This diverse training data is essential for instruction-based finetuning, which enables the embedding model to learn from a wide range of tasks and domains.'

In [ ]:
gold_ans2_mkt=validation_questions_answers[50]['gold_answer_marketing']

In [ ]:
gold_ans2_mkt

'Training data for embedding models that use task-specific instructions is typically created by formulating a wide variety of tasks as text-to-text problems, distinguishing good/bad candidate outputs given an input text. This is done by combining datasets with natural language instructions and constructing positive and negative pairs for training.'

a) The RAG results (research and marketing response)

In [ ]:
test_ans2_eng = final_rag_pipe_eng.invoke(test_q2)

In [ ]:
test_ans2_eng

'A common approach is to construct large-scale datasets with diverse tasks and human-written instructions, such as the MEDI dataset, which combines datasets from SuperGLUE and creates additional instructions for tasks. This provides a wide range of contexts and enables models to learn from explicit directives. Another strategy is to use a contrastive loss function during training to encourage embeddings to be similar for inputs with the same instructions and dissimilar for those with different instructions.'

In [ ]:
test_ans2_mkt = final_rag_pipe_mkt.invoke(test_q2)

In [ ]:
test_ans2_mkt

"Training data for embedding models that utilize task-specific instructions is often created by combining large datasets from multiple sources and tasks, and then annotating these datasets with human-written instructions. This approach provides a diverse and comprehensive training mixture, enhancing the model's performance on various embedding tasks."

b) The context provided


In [ ]:
final_rag_context_provided(test_q2)

'variety of tasks for embedding training with in-\nstructions. We thus construct a collection of 330\ndatasets with instructions across diverse task cate-\ngories and domains: Multitask Embeddings Data\nwith Instructions (MEDI).\nData\nConstruction\nWe\nbuild\nMEDI\nby\ncombining\n300\ndatasets\nfrom\nSuper-\n\ncontext learning. INSTRUCTOR significantly out-\nperforms prior state-of-the-art embedding models\nby an average of 3.4% over the 70 diverse datasets.\nINSTRUCTOR also outperforms a variant that is\ntrained without task instructions (§4), demonstrat-\ning the importance of instructions to create task-\n\nmultitask mixture of 330 diverse datasets with human-written task instructions (MEDI dataset, §2.3). After training\non MEDI (left), INSTRUCTOR is evaluated on a variety of 70 embedding datasets (66 of which are not seen during\n\nfurther training. We first annotate instructions\nfor 330 diverse tasks and train INSTRUCTOR\non this multitask mixture with a contrastive loss.\nWe e

c) The document sources for the context


In [ ]:
get_document_sources(test_q2)

document:  variety of tasks for embedding training with in-
structions. We thus construct a collection of 330
datasets with instructions across diverse task cate-
gories and domains: Multitask Embeddings Data
with Instructions (MEDI).
Data
Construction
We
build
MEDI
by
combining
300
datasets
from
Super-
source:  https://arxiv.org/pdf/2212.09741.pdf
doc_source: ArXiv


document:  context learning. INSTRUCTOR significantly out-
performs prior state-of-the-art embedding models
by an average of 3.4% over the 70 diverse datasets.
INSTRUCTOR also outperforms a variant that is
trained without task instructions (§4), demonstrat-
ing the importance of instructions to create task-
source:  https://arxiv.org/pdf/2212.09741.pdf
doc_source: ArXiv


document:  multitask mixture of 330 diverse datasets with human-written task instructions (MEDI dataset, §2.3). After training
on MEDI (left), INSTRUCTOR is evaluated on a variety of 70 embedding datasets (66 of which are not seen during
source:  https:/

d) Also discuss your metric(s) for the first two examples (for both responses) compared to the gold responses

In [ ]:
eng_cos_sim2=get_cosine_similarity(gold_ans2_eng,test_ans2_eng)
eng_bert_score2=get_bert_score(gold_ans2_eng,test_ans2_eng)
eng_combined_score2=(eng_cos_sim2+eng_bert_score2['f1'])/2

mkt_cos_sim2=get_cosine_similarity(gold_ans2_mkt,test_ans2_mkt)
mkt_bert_score2=get_bert_score(gold_ans2_mkt,test_ans2_mkt)
mkt_combined_score2=(mkt_cos_sim2+mkt_bert_score2['f1'])/2

In [ ]:
print('Engineering Evaluation:')
print('cosine similarity:', eng_cos_sim2)
print('bert score:', eng_bert_score2)
print('combined score:', eng_combined_score2)
print('\n\n')

print('Marketing Evaluation:')
print('cosine similarity:', mkt_cos_sim2)
print('bert score:', mkt_bert_score2)
print('combined score:', mkt_combined_score2)

Engineering Evaluation:
cosine similarity: 0.7304141479983661
bert score: {'precision': 0.6751267313957214, 'recall': 0.6467127799987793, 'f1': 0.6606143116950989}
combined score: 0.6955142298467325



Marketing Evaluation:
cosine similarity: 0.9198096650898755
bert score: {'precision': 0.7299734950065613, 'recall': 0.6987491250038147, 'f1': 0.7140201330184937}
combined score: 0.8169148990541846


Discuss...

#### 5.2.3 Test Question 3

Please run the query:

In [ ]:
test_q3=test_questions[83]['question']

a) The RAG results (research and marketing response)

In [ ]:
test_ans3_eng = final_rag_pipe_eng.invoke(test_q3)

In [ ]:
test_ans3_eng

"A model's ability to answer questions is directly influenced by its exposure to specific question types during training. If a model encounters a question during testing that is identical to one it has seen during training, it can often provide the correct answer through memorization. Models can also select answers from a set of trained responses, even if the question is novel. However, the true test of a model's capabilities is its ability to answer questions with responses that were not explicitly provided in the training data, requiring more complex reasoning and understanding."

In [ ]:
test_ans3_mkt = final_rag_pipe_mkt.invoke(test_q3)

In [ ]:
test_ans3_mkt

"A model's ability to answer questions is directly influenced by its exposure to similar questions during training. Models can only provide answers from their training data, so novel questions at test time challenge the model to choose the best response from its limited knowledge base. More complex tasks, like multi-hop QA, require models to reason and interpret, showcasing a higher level of understanding."

b) The context provided


In [ ]:
final_rag_context_provided(test_q3)

'A model is able to correctly memorize and respond with the answer to a question that has been seen at training time.\nA model is able to answer novel questions at test time and choose an answer from the set of answers it has seen during training.\n\nA model is able to answer novel questions which have answers not contained in the training dataset.\n\n(EM) metric to evaluate model predictions.\nMulti-Hop Question Answering\nWe also adopt\nmore complex QA scenarios, the first of which is\nmulti-hop QA, where each question q requires rea-\nsoning over a chain of passages P to obtain the\ncorrect answer o. For this task, we use the Hot-\n\nbetween questions in the train and test sets in several public QA datasets.'

c) The document sources for the context


In [ ]:
get_document_sources(test_q3)

document:  A model is able to correctly memorize and respond with the answer to a question that has been seen at training time.
A model is able to answer novel questions at test time and choose an answer from the set of answers it has seen during training.
source:  https://lilianweng.github.io/posts/2020-10-29-odqa/
doc_source: WWW


document:  A model is able to answer novel questions which have answers not contained in the training dataset.
source:  https://lilianweng.github.io/posts/2020-10-29-odqa/
doc_source: WWW


document:  (EM) metric to evaluate model predictions.
Multi-Hop Question Answering
We also adopt
more complex QA scenarios, the first of which is
multi-hop QA, where each question q requires rea-
soning over a chain of passages P to obtain the
correct answer o. For this task, we use the Hot-
source:  https://arxiv.org/pdf/2311.08377.pdf
doc_source: ArXiv


document:  between questions in the train and test sets in several public QA datasets.
source:  https://lilianweng.

Discuss...

### 5.3 Other Questions

Below are a few questions that you should think about. Please answer them in the answer file directly (in a short paragraph) and also see whether they may be relevant for your final write-up.

**QUESTION:**

5.3.a. How would you expect your response quality to change if you had a chunk size of 50?

- If I had a chunk size of 50, I would expect the RAG to provide lower quality responses. With such a small chunk size, there would be more fragmented answers. I also think that the smaller chunks might result in a lot of missed information in the document sources that could be useful for providing relevant responses. This could result in responses that lack the depth and comprehensiveness needed to respond to the questions well.

5.3.b. How would you expect your response quality to change if you had a chunk size of 5000?
- If I had a chunk size of 5000, I would expect the RAG to provide lower quality answers. With such a large chunk size, there would be too much information overloading the system, and responses might contain irrelevant details unrelated to the question at hand. Overall, there would likely be less clarity and relevance in responses with such large chunk sizes.


5.3.c. If you had time, how do you think fine-tuning of the LLM could help?  What type of data would you want for that? And which training approach would you take?
- Fine-tuning the LLM could help by improving the relevance of answers and reducing hallucination, given that the data provided is high quality and the training method is appropriate. Speaking of, best data for this fine-tuning task might be pairs of queries and appropriate model responses. For the training approach, I would utilize instruction tuning with the aforementioned labeled datapoints of (prompt, response) pairs, and I would supplement this with Reinforcement learning from human feedback (RLHF), which would seamlessly integrate human feedback from chat conversations.


5.3.d. What was your design philosophy  of the prompts? How did they differ between engineering and marketing support?
- I designed prompts that were clear indications of the instructions, that repeated the instructions multiple times in different words to push for appropriate responses. The prompts for  engineering support expressed that the audience was already highly technical and knowledgeable in the fundamentals of NLP, so they needed more technical and detailed responses. The prompts for marketing support expressed that the audience was not technical and needed clear and high-level responses that didn't involve too much technical detail. I also asked for a slightly longer answer for engineering than for marketing.

5.3.e. What are your average and peak load estimates for the system? Given that, would you suggest a pay-per-use deployment or one that reserves the LLM?
- My average load estimate for the system is about 10 queries a day per marketing staff and engineering staff. However, there may be scenarious such as deadlines or campaigns that would spike usage, which leads me to estimate peak loads to be 25 queries, with perhaps 50% of the staff being concurrent users. Overall, given my estimates, I would suggest reseerving the LLM because in such a dynamic working environment the system will likely consistently have high usage and will need to handle peaks regularly.

5.3.f. What type of limitations/risks would you see in using this system?
- The first risk in using this system is the potential for inaccurate answers or unhelpful answers, which could be attributable to faults in RAG system specifications or faults in the documents that the system has access to. Another limitation has to do with scalability: the computational and financial costs associated with deploying and maintaining such a system at a large scale could be difficult to expend given company resources. Finally, risks related to security may arise because the model may gain access to confidential company information, and a data breach may occur.